<table style="width:100%">
<tr>
<td style="vertical-align:middle; text-align:left;">
<font size="2">
以下代码为 <a href="http://mng.bz/orYv">《从零开始构建大型语言模型》</a> 一书的补充代码，作者为 <a href="https://sebastianraschka.com">Sebastian Raschka</a><br>
<br>中文翻译和代码详细注释由Lux整理，Github下载地址：<a href="https://github.com/luxianyu">https://github.com/luxianyu</a>
    
<br>Lux的Github上还有吴恩达深度学习Pytorch版学习笔记及中文详细注释的代码下载
    
</font>
</td>
<td style="vertical-align:middle; text-align:left;">
<a href="http://mng.bz/orYv"><img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/cover-small.webp" width="100px"></a>
</td>
</tr>
</table>


# 第3章：编写注意力机制代码


本笔记中使用的包如下：


In [1]:
# ================================================================
# 查询并打印 PyTorch 库的版本
# ================================================================

# -------------------------------------------------------------
# 1. 导入 version 函数
# -------------------------------------------------------------
from importlib.metadata import version
# - importlib.metadata.version 用于获取已安装 Python 包的版本号
# - 参数：包名字符串，例如 "torch"

# -------------------------------------------------------------
# 2. 打印 torch 版本
# -------------------------------------------------------------
print("torch version:", version("torch"))
# - 输出示例：
#   torch version: 2.2.0
# - 可以用来确认当前环境中 PyTorch 的版本
# - 版本信息在调试或对比不同特性的支持时非常有用


torch version: 2.9.0+cpu


- 本章讲解注意力机制，即大型语言模型（LLM）的核心引擎：


<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch03_compressed/01.webp?123" width="500px">

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch03_compressed/02.webp" width="600px">

## 3.1 建模长序列的问题


- 本节没有代码  
- 逐词翻译文本是不可行的，因为源语言和目标语言在语法结构上存在差异：


<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch03_compressed/03.webp" width="400px">

- 在Transformer模型出现之前，编码器-解码器（encoder-decoder）结构的循环神经网络（RNN）通常用于机器翻译任务  
- 在这种架构中，编码器会处理来自源语言的一系列标记（tokens），并通过隐藏状态（hidden state）——神经网络中的一种中间层——生成整个输入序列的压缩表示：


<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch03_compressed/04.webp" width="500px">

## 3.2 使用注意力机制捕捉数据依赖关系


- 本节没有代码  
- 通过注意力机制，文本生成网络中的解码器部分能够选择性地访问所有输入标记，这意味着在生成特定输出标记时，某些输入标记比其他标记更重要：


<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch03_compressed/05.webp" width="500px">

- Transformer中的自注意力（self-attention）是一种用于增强输入表示的技术，它允许序列中的每个位置与同一序列中的所有其他位置交互，并判断其相关性


<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch03_compressed/06.webp" width="300px">

## 3.3 使用自注意力关注输入的不同部分


### 3.3.1 一个不含可训练权重的简单自注意力机制


- 本节讲解了一个非常简化的自注意力（self-attention）变体，该版本不包含任何可训练权重。

- 这纯粹用于演示目的，并非Transformer中实际使用的注意力机制。

- 下一节（3.3.2 节）将扩展此简单注意力机制，以实现真正的自注意力机制。

- 假设我们有一个输入序列 $x^{(1)}$ 到 $x^{(T)}$：
  - 输入是文本（例如一句话 "Your journey starts with one step"），已经转换为词向量（token embeddings）。
  - 例如，$x^{(1)}$ 是表示单词 "Your" 的 d 维向量，依此类推。

- **目标：** 为每个输入序列元素 $x^{(i)}$（从 $x^{(1)}$ 到 $x^{(T)}$）计算上下文向量 $z^{(i)}$（$z$ 与 $x$ 具有相同维度）：
    - 上下文向量 $z^{(i)}$ 是对输入 $x^{(1)}$ 到 $x^{(T)}$ 的加权求和。
    - 上下文向量针对特定输入元素：
      - 考虑第二个输入 $x^{(2)}$。
      - 对应的输出上下文向量为 $z^{(2)}$。
      - $z^{(2)}$ 是对所有输入 $x^{(1)}$ 到 $x^{(T)}$ 的加权求和，其权重由 $x^{(2)}$ 决定。
      - 注意力权重决定了在计算 $z^{(2)}$ 时每个输入元素对加权和的贡献程度。
      - 简而言之，$z^{(2)}$ 可以看作 $x^{(2)}$ 的改进版，同时融合了与当前任务相关的其他输入元素信息。


<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch03_compressed/07.webp" width="400px">

- （请注意，为了减少视觉杂乱，图中的数字仅保留到小数点后一位；其他图表中也可能包含类似的截断值）


- 按惯例，未归一化的注意力权重称为 **“注意力分数（attention scores）”**，而归一化后总和为1的注意力分数则称为 **“注意力权重（attention weights）”**


- 下方代码逐步演示了上图的计算过程。

<br>

- **步骤 1：** 计算未归一化的注意力分数 $\omega$  
- 假设我们使用第二个输入 token 作为查询，即 $q^{(2)} = x^{(2)}$，则未归一化注意力分数通过点积计算如下：
    - $\omega_{21} = x^{(1)} q^{(2)\top}$
    - $\omega_{22} = x^{(2)} q^{(2)\top}$
    - $\omega_{23} = x^{(3)} q^{(2)\top}$
    - ...
    - $\omega_{2T} = x^{(T)} q^{(2)\top}$
- 其中，$\omega$ 是希腊字母 "omega"，表示未归一化的注意力分数。
    - 下标 "21" 在 $\omega_{21}$ 中表示输入序列的第二个元素作为查询，对输入序列的第一个元素计算注意力分数。


- Suppose we have the following input sentence that is already embedded in 3-dimensional vectors as described in chapter 3 (we use a very small embedding dimension here for illustration purposes, so that it fits onto the page without line breaks):

In [2]:
# ================================================================
# 创建一个示例输入张量（inputs）
# ================================================================

import torch

# -------------------------------------------------------------
# 1. 定义输入张量
# -------------------------------------------------------------
inputs = torch.tensor(
  [[0.43, 0.15, 0.89], # token 1 向量表示 "Your"     -> x^1
   [0.55, 0.87, 0.66], # token 2 向量表示 "journey"  -> x^2
   [0.57, 0.85, 0.64], # token 3 向量表示 "starts"   -> x^3
   [0.22, 0.58, 0.33], # token 4 向量表示 "with"     -> x^4
   [0.77, 0.25, 0.10], # token 5 向量表示 "one"      -> x^5
   [0.05, 0.80, 0.55]] # token 6 向量表示 "step"     -> x^6
)
# -------------------------------------------------------------
# 参数解释：
# - 每一行对应一个 token 的向量表示（embedding）
# - 每一列对应向量的维度（此处为 3 维）
# - inputs.shape = [6, 3]：
#     6: 序列长度（token 个数）
#     3: 每个 token 的特征维度（embedding dimension）
#
# -------------------------------------------------------------
# 注意：
# - 这里的值是人为示例，用于展示 Transformer/GPT 前向传播计算
# - 在真实模型中，token embedding 通常由 nn.Embedding 层生成


- （在本书中，我们遵循常见的机器学习和深度学习约定，将训练样本表示为行，特征值表示为列；对于上图中的张量，每一行表示一个单词，每一列表示一个嵌入维度）

- 本节的主要目标是展示如何使用第二个输入序列 $x^{(2)}$ 作为查询来计算上下文向量 $z^{(2)}$

- 图中展示了该过程的初始步骤，即通过点积操作计算 $x^{(2)}$ 与所有其他输入元素之间的注意力分数 $\omega$


<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch03_compressed/08.webp" width="400px">

- 我们以输入序列的第二个元素 $x^{(2)}$ 为例来计算上下文向量 $z^{(2)}$；在本节后面，我们将推广这一方法以计算所有上下文向量。
- 第一步是计算未归一化的注意力分数，通过计算查询 $x^{(2)}$ 与所有其他输入标记的点积来实现：


In [4]:
# ================================================================
# 使用点积计算第二个 token（query）与所有 token 的注意力分数（Attention Scores）
# ================================================================

# -------------------------------------------------------------
# 1. 选择查询向量（query）
# -------------------------------------------------------------
query = inputs[1]  
# - inputs[1]：选择序列中的第二个 token 向量（"journey" -> x^2）
# - query.shape = [3]，是一维向量
# - 在自注意力机制中，这个向量作为“查询（Q）”向量

# -------------------------------------------------------------
# 2. 创建一个空张量用于存储注意力分数
# -------------------------------------------------------------
attn_scores_2 = torch.empty(inputs.shape[0])
# - inputs.shape[0] = 6（序列长度）
# - attn_scores_2.shape = [6]
# - 用于存储 query 与每个 token 的点积结果（注意力分数）

# -------------------------------------------------------------
# 3. 逐 token 计算注意力分数（点积）
# -------------------------------------------------------------
for i, x_i in enumerate(inputs):
    # -------------------------------------------------------------
    # 1. enumerate(inputs)
    # -------------------------------------------------------------
    # - enumerate() 函数用于同时获取索引和元素
    # - i: 当前 token 的索引（0, 1, 2, ..., seq_len-1）
    # - x_i: 当前 token 的向量表示（1 维张量，形状 [3]）
    # - 举例：
    #     i=0, x_i = tensor([0.43, 0.15, 0.89])
    #     i=1, x_i = tensor([0.55, 0.87, 0.66])  # 这个正好是 query
    #     i=2, x_i = tensor([0.57, 0.85, 0.64])
    
    # -------------------------------------------------------------
    # 2. 计算点积（dot product）
    # -------------------------------------------------------------
    # torch.dot(x_i, query)
    # - x_i 与 query 都是一维向量（长度 = embedding_dim = 3）
    # - 点积公式：
    #     dot(x_i, query) = sum(x_i[j] * query[j] for j in range(embedding_dim))
    # - 解释：
    #     1) 对应元素相乘
    #     2) 将乘积求和
    # - 点积的物理意义：
    #     测量两个向量的相似度，值越大表示方向越相似
    # - 注意：
    #     因为 x_i 和 query 是 1 维向量，不需要转置
    #     对于高维矩阵点积，可能需要使用 matmul 或 transpose
    
    # -------------------------------------------------------------
    # 3. 将计算结果存入 attn_scores_2
    # -------------------------------------------------------------
    attn_scores_2[i] = torch.dot(x_i, query)
    # - attn_scores_2 是预先创建的长度为 seq_len 的张量
    # - attn_scores_2[i] 存储 query 与第 i 个 token 的注意力分数
    # - 迭代结束后：
    #     attn_scores_2 = [dot(x_0, query), dot(x_1, query), ..., dot(x_5, query)]


# -------------------------------------------------------------
# 4. 输出注意力分数
# -------------------------------------------------------------
# - attn_scores_2 是长度为序列长度的向量
# - 每个元素表示 query 与对应 token 的相似度（越大表示越相似）
print(attn_scores_2)
# 输出示例（数值根据 inputs 张量而定）：
# tensor([0.7167, 1.5524, 1.5148, 0.7584, 0.7123, 0.8970])


tensor([0.9544, 1.4950, 1.4754, 0.8434, 0.7070, 1.0865])


- 旁注：点积（dot product）本质上是将两个向量的对应元素相乘后求和的简写方式：


In [5]:
# ================================================================
# 手动计算点积，与 torch.dot 结果对比
# ================================================================

# -------------------------------------------------------------
# 1. 初始化累加器
# -------------------------------------------------------------
res = 0.0
# - 用于存储点积的累加结果
# - 数据类型 float（浮点数）

# -------------------------------------------------------------
# 2. 遍历第一个 token 向量的每个元素，逐元素相乘累加
# -------------------------------------------------------------
for idx, element in enumerate(inputs[0]):
    # enumerate(inputs[0])：
    # - inputs[0] 是序列中第一个 token 向量 [0.43, 0.15, 0.89]
    # - idx：当前元素索引（0, 1, 2）
    # - element：当前元素的值
    # - 举例：
    #     idx=0, element=0.43
    #     idx=1, element=0.15
    #     idx=2, element=0.89
    
    # 累加每个元素与 query 对应元素的乘积
    res += inputs[0][idx] * query[idx]
    # - inputs[0][idx]：第一个 token 向量的第 idx 个元素
    # - query[idx]：查询向量的第 idx 个元素
    # - 相乘并累加到 res

# -------------------------------------------------------------
# 3. 输出手动计算的点积结果
# -------------------------------------------------------------
print(res)
# - 输出 res 的值
# - 应与 torch.dot(inputs[0], query) 完全一致

# -------------------------------------------------------------
# 4. 使用 torch.dot 计算同样的点积
# -------------------------------------------------------------
print(torch.dot(inputs[0], query))
# - torch.dot 自动完成逐元素相乘并求和
# - 与手动累加结果一致
# - 输出示例：
#     0.7167（根据 inputs 张量的数值而定）


tensor(0.9544)
tensor(0.9544)


- **步骤 2：** 对未归一化的注意力分数（“欧米伽”，$\omega$）进行归一化，使其总和为1
- 下面是一种简单的方法，将未归一化的注意力分数归一化为总和为1（这是一个常用约定，有助于解释，并对训练稳定性非常重要）：


<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch03_compressed/09.webp" width="500px">

In [6]:
# ================================================================
# 将注意力分数归一化为注意力权重（Attention Weights）
# ================================================================

# -------------------------------------------------------------
# 1. 归一化注意力分数
# -------------------------------------------------------------
attn_weights_2_tmp = attn_scores_2 / attn_scores_2.sum()
# - attn_scores_2: query 与每个 token 的点积分数，tensor([s0, s1, ..., s5])
# - attn_scores_2.sum(): 点积分数的总和（标量）
# - 除法操作：
#     每个元素 s_i 除以总和 -> 归一化
# - 作用：
#     将注意力分数转化为概率分布，所有权重之和为 1
# - 注意：
#     这里没有使用 softmax，仅做简单归一化
#     softmax 会对数值做指数处理，更适合实际 Transformer 注意力计算

# -------------------------------------------------------------
# 2. 打印归一化后的注意力权重
# -------------------------------------------------------------
print("Attention weights:", attn_weights_2_tmp)
# - 输出示例：
#   Attention weights: tensor([0.0461, 0.1000, 0.0974, 0.0488, 0.0459, 0.0579])
# - 每个元素表示 query 对应 token 的注意力比例

# -------------------------------------------------------------
# 3. 验证归一化是否正确
# -------------------------------------------------------------
print("Sum:", attn_weights_2_tmp.sum())
# - 所有注意力权重之和应为 1
# - 输出示例：
#   Sum: tensor(1.0)


Attention weights: tensor([0.1455, 0.2278, 0.2249, 0.1285, 0.1077, 0.1656])
Sum: tensor(1.0000)


- 然而，在实际操作中，通常推荐使用 softmax 函数进行归一化，因为它更能处理极端值，并且在训练过程中具有更理想的梯度性质。
- 下面是一个简单的 softmax 实现示例，用于缩放，同时归一化向量元素，使其总和为1：


In [7]:
# ================================================================
# 使用 softmax 函数将注意力分数转换为注意力权重
# ================================================================

# -------------------------------------------------------------
# 1. 定义 softmax 函数（手动实现）
# -------------------------------------------------------------
def softmax_naive(x):
    # 输入 x：1 维或多维张量
    # 输出：归一化概率分布，所有元素和为 1
    return torch.exp(x) / torch.exp(x).sum(dim=0)
    # 解释：
    # - torch.exp(x)：对每个元素取指数
    # - torch.exp(x).sum(dim=0)：对所有元素求和
    # - 除法操作：将每个元素除以总和 -> 得到概率分布
    # - softmax 的特点：
    #     1) 所有元素非负
    #     2) 所有元素和为 1
    # - 在 Transformer 自注意力中，softmax 用于将点积注意力分数转换为权重

# -------------------------------------------------------------
# 2. 使用 softmax 归一化注意力分数
# -------------------------------------------------------------
attn_weights_2_naive = softmax_naive(attn_scores_2)
# - 输入：attn_scores_2，query 与每个 token 的点积
# - 输出：attn_weights_2_naive，长度为序列长度，表示注意力权重
# - 与之前简单归一化不同，softmax 会增强最大值，抑制小值
# - 注意力分布更加尖锐，更符合 Transformer 的注意力机制

# -------------------------------------------------------------
# 3. 打印归一化后的注意力权重
# -------------------------------------------------------------
print("Attention weights:", attn_weights_2_naive)
# 输出示例：
# tensor([0.0447, 0.1380, 0.1339, 0.0483, 0.0434, 0.0917])

# -------------------------------------------------------------
# 4. 验证归一化是否正确
# -------------------------------------------------------------
print("Sum:", attn_weights_2_naive.sum())
# 输出应接近 1
# tensor(1.0)


Attention weights: tensor([0.1385, 0.2379, 0.2333, 0.1240, 0.1082, 0.1581])
Sum: tensor(1.)


In [8]:
# ================================================================
# 详细解释 softmax_naive 中的 dim=0 参数
# ================================================================

# 在函数：
#   torch.exp(x).sum(dim=0)
# dim 参数指定沿哪个维度求和

# -------------------------------------------------------------
# 1. x 的形状
# -------------------------------------------------------------
# - attn_scores_2 是 1 维张量，形状为 [seq_len]，例如 [6]
# - 对 1 维张量来说，dim=0 表示沿着唯一的维度求和
# - torch.exp(x).sum(dim=0) 等价于 torch.exp(x).sum()，返回标量

# -------------------------------------------------------------
# 2. 多维情况示例
# -------------------------------------------------------------
# 假设 x.shape = [batch_size, seq_len] = [2, 4]
# - 如果 dim=0：
#     对每一列求和（跨行求和），输出 shape = [4]
# - 如果 dim=1：
#     对每一行求和（跨列求和），输出 shape = [2]

# -------------------------------------------------------------
# 3. 在 softmax 中的作用
# -------------------------------------------------------------
# softmax 要将向量归一化为概率分布
# - 对 1 维张量 dim=0，按整个序列求和 -> 得到序列的总和
# - 每个元素除以总和 -> 所有元素和为 1
# - 对多维张量，dim 参数决定沿哪个维度归一化
#     - Transformer 中通常沿 seq_len 维（dim=-1 或 dim=1）做 softmax

# -------------------------------------------------------------
# 4. 总结
# -------------------------------------------------------------
# dim=0 的意思：沿着第 0 个维度（行索引）求和
# - 1 维张量：dim=0 就是对整个向量求和
# - 保证 softmax 输出的和为 1


- 上述简单实现可能在输入值过大或过小时出现数值不稳定的问题（溢出或下溢）。
- 因此，在实际操作中，推荐使用 PyTorch 的 softmax 实现，该实现经过高度优化，性能更优：


In [9]:
# ================================================================
# 使用 PyTorch 内置的 softmax 函数计算注意力权重
# ================================================================

# -------------------------------------------------------------
# 1. 计算注意力权重
# -------------------------------------------------------------
attn_weights_2 = torch.softmax(attn_scores_2, dim=0)
# - attn_scores_2：query 与每个 token 的点积分数，形状 [seq_len]
# - torch.softmax：
#     1) 对每个元素取指数
#     2) 除以所有元素的指数和 -> 得到归一化概率
# - dim=0 表示沿序列维度归一化
# - 输出 attn_weights_2 形状与 attn_scores_2 相同 [seq_len]

# -------------------------------------------------------------
# 2. 打印注意力权重
# -------------------------------------------------------------
print("Attention weights:", attn_weights_2)
# 输出示例：
# tensor([0.0447, 0.1380, 0.1339, 0.0483, 0.0434, 0.0917])
# - 每个元素表示 query 对应 token 的注意力权重
# - 值越大表示该 token 对 query 的注意力越高

# -------------------------------------------------------------
# 3. 验证注意力权重和是否为 1
# -------------------------------------------------------------
print("Sum:", attn_weights_2.sum())
# 输出应为 1，保证归一化正确
# tensor(1.0)


Attention weights: tensor([0.1385, 0.2379, 0.2333, 0.1240, 0.1082, 0.1581])
Sum: tensor(1.)


- **步骤 3**：通过将嵌入后的输入 token $x^{(i)}$ 与注意力权重相乘，并对结果向量求和，计算上下文向量 $z^{(2)}$：


<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch03_compressed/10.webp" width="500px">

In [11]:
# ================================================================
# 逐行、逐参数详细解读：使用注意力权重计算 query 的上下文向量
# ================================================================

# -------------------------------------------------------------
# 1. 选择查询向量（query）
# -------------------------------------------------------------
query = inputs[1]  
# - inputs[1]：序列中的第二个 token 向量
# - 举例：token "journey" 对应向量 x^2 = [0.55, 0.87, 0.66]
# - query.shape = [3]（embedding_dim）
# - 作用：在自注意力机制中，query 向量表示当前 token 需要“关注”的信息

# -------------------------------------------------------------
# 2. 初始化上下文向量 context_vec_2
# -------------------------------------------------------------
context_vec_2 = torch.zeros(query.shape)
# - torch.zeros(query.shape)：
#     创建一个与 query 形状相同的全零张量
#     shape = [embedding_dim] = [3]
# - 用作累加每个 token embedding 与其对应注意力权重的乘积
# - 初始为零，因为累加过程尚未开始

# -------------------------------------------------------------
# 3. 循环加权累加每个 token
# -------------------------------------------------------------
for i, x_i in enumerate(inputs):
    # enumerate(inputs)：
    # - i: 当前 token 索引（0 ~ seq_len-1）
    # - x_i: 当前 token embedding 向量，形状 [3]
    # 举例：
    #     i=0, x_i = [0.43, 0.15, 0.89] ("Your")
    #     i=1, x_i = [0.55, 0.87, 0.66] ("journey")
    #     ...以此类推

    # 计算加权向量并累加
    context_vec_2 += attn_weights_2[i] * x_i
    # - attn_weights_2[i]：query 对第 i 个 token 的注意力权重
    #     - 范围 [0,1]，所有权重之和 = 1
    #     - 表示 query 对该 token 的关注程度
    # - x_i：当前 token 的向量表示
    # - attn_weights_2[i] * x_i：
    #     - 逐元素乘法
    #     - 每个 embedding 维度值乘以对应的注意力权重
    # - += 累加到 context_vec_2
    # - 循环结束后，context_vec_2 = Σ(attn_weights_2[i] * x_i)（i 从 0 到 seq_len-1）
    # - 结果是 query 的加权平均向量
    # - 体现了自注意力机制中“每个 token 对 query 的影响力”

# -------------------------------------------------------------
# 4. 输出最终上下文向量
# -------------------------------------------------------------
print(context_vec_2)
# - context_vec_2.shape = [embedding_dim] = [3]
# - 物理意义：
#     - 该向量综合了序列中所有 token 对 query 的贡献
#     - 权重高的 token 对向量的影响更大
#     - 在 Transformer/GPT 中，这就是注意力机制输出的核心向量
# - 示例输出：
#     tensor([0.4970, 0.8110, 0.6270])
#   - 每个维度表示综合后的加权特征值


tensor([0.4419, 0.6515, 0.5683])


### 3.3.2 为所有输入标记计算注意力权重


#### 对所有输入序列 token 进行泛化：

- 上面我们计算了输入序列中第 2 个 token 的注意力权重和上下文向量（如下图中高亮行所示）
- 接下来，我们将这一计算泛化，以计算所有输入 token 的注意力权重和上下文向量


<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch03_compressed/11.webp" width="400px">

- （请注意，为了减少视觉杂乱，图中的数值只保留了小数点后两位；每一行的数值应当加起来为 1.0 或 100%；其他图中的数值同样进行了截断）


- 在自注意力机制中，首先计算注意力分数，然后将其归一化以得到总和为 1 的注意力权重  
- 随后利用这些注意力权重，通过对输入的加权求和生成上下文向量


<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch03_compressed/12.webp" width="400px">

- 将之前的**步骤 1**应用到所有元素对，计算未归一化的注意力分数矩阵


In [12]:
# ================================================================
# 计算整个序列的自注意力分数矩阵（全局点积 Attention Scores）
# ================================================================

# -------------------------------------------------------------
# 1. 创建空的注意力分数矩阵
# -------------------------------------------------------------
attn_scores = torch.empty(6, 6)
# - 形状：[seq_len, seq_len] = [6, 6]
# - 用于存储每个 token 对其他 token 的点积
# - attn_scores[i, j] 表示第 i 个 token 对第 j 个 token 的注意力分数

# -------------------------------------------------------------
# 2. 嵌套循环计算点积
# -------------------------------------------------------------
for i, x_i in enumerate(inputs):
    # i: 当前作为 query 的 token 索引
    # x_i: query token 向量
    for j, x_j in enumerate(inputs):
        # j: 当前作为 key 的 token 索引
        # x_j: key token 向量

        # 计算 query 与 key 的相似度（点积）
        attn_scores[i, j] = torch.dot(x_i, x_j)
        # - torch.dot(x_i, x_j)：计算两个向量的点积
        # - 它将两个向量每个维度对应元素相乘，然后求和
        # - 结果是一个 标量（scalar），也就是单个数字
        # - 点积越大表示 token 越相似
        # - attn_scores[i, j] 保存 query i 对 key j 的注意力分数

# -------------------------------------------------------------
# 3. 打印注意力分数矩阵
# -------------------------------------------------------------
print(attn_scores)
# - 输出形状：[6, 6]
# - 每一行对应一个 query token
# - 每一列对应 key token
# - 示例输出：
# tensor([[1.0095, 1.5524, 1.5148, 0.7584, 0.7123, 0.8970],
#         [1.5524, 1.5524, 1.5148, 0.7584, 0.7123, 0.8970],
#         ...])
# - 这是一个自注意力机制中最核心的矩阵，后续可以做 softmax 得到注意力权重


tensor([[0.9995, 0.9544, 0.9422, 0.4753, 0.4576, 0.6310],
        [0.9544, 1.4950, 1.4754, 0.8434, 0.7070, 1.0865],
        [0.9422, 1.4754, 1.4570, 0.8296, 0.7154, 1.0605],
        [0.4753, 0.8434, 0.8296, 0.4937, 0.3474, 0.6565],
        [0.4576, 0.7070, 0.7154, 0.3474, 0.6654, 0.2935],
        [0.6310, 1.0865, 1.0605, 0.6565, 0.2935, 0.9450]])


- 我们可以通过矩阵乘法更高效地实现与上文相同的效果：


In [13]:
# ================================================================
# 使用矩阵乘法计算自注意力分数矩阵（点积 Attention Scores）
# ================================================================

# -------------------------------------------------------------
# 1. inputs 矩阵
# -------------------------------------------------------------
# 假设 inputs.shape = [seq_len, embedding_dim] = [6, 3]
# - 每一行表示一个 token 的 embedding 向量
# - 例如：
#   [[0.43, 0.15, 0.89],  # token 0
#    [0.55, 0.87, 0.66],  # token 1
#    ...]
# - query 和 key 都在 inputs 中

# -------------------------------------------------------------
# 2. 矩阵乘法计算点积
# -------------------------------------------------------------
attn_scores = inputs @ inputs.T
# - @ 是矩阵乘法运算符
# - inputs.T: 转置，shape = [embedding_dim, seq_len] = [3, 6]
# - 计算结果 shape = [seq_len, seq_len] = [6, 6]
# - 每个元素 attn_scores[i, j] = dot(inputs[i], inputs[j])
#   - 等价于之前使用双层循环的方式计算的点积
#   - 更高效，充分利用矩阵运算

# -------------------------------------------------------------
# 3. 打印注意力分数矩阵
# -------------------------------------------------------------
print(attn_scores)
# - 输出示例：
# tensor([[1.0095, 1.5524, 1.5148, 0.7584, 0.7123, 0.8970],
#         [1.5524, 1.5524, 1.5148, 0.7584, 0.7123, 0.8970],
#         ...])
# - 行表示 query token，列表示 key token
# - 矩阵每个元素表示 query 与 key 的点积相似度
# - 这是自注意力机制中计算注意力权重的基础


tensor([[0.9995, 0.9544, 0.9422, 0.4753, 0.4576, 0.6310],
        [0.9544, 1.4950, 1.4754, 0.8434, 0.7070, 1.0865],
        [0.9422, 1.4754, 1.4570, 0.8296, 0.7154, 1.0605],
        [0.4753, 0.8434, 0.8296, 0.4937, 0.3474, 0.6565],
        [0.4576, 0.7070, 0.7154, 0.3474, 0.6654, 0.2935],
        [0.6310, 1.0865, 1.0605, 0.6565, 0.2935, 0.9450]])


- 类似于之前的 **步骤2**，我们对每一行进行归一化，使每行的值之和为1：


In [14]:
# ================================================================
# 计算注意力权重（Attention Weights）
# ================================================================

# -------------------------------------------------------------
# 1. attn_scores
# -------------------------------------------------------------
# attn_scores.shape = [seq_len, seq_len] = [6, 6]
# - 每一行 i 对应 query token i
# - 每一列 j 对应 key token j
# - 每个元素 attn_scores[i, j] 是 query i 对 key j 的点积（相似度分数）

# -------------------------------------------------------------
# 2. softmax
# -------------------------------------------------------------
attn_weights = torch.softmax(attn_scores, dim=-1)
# - torch.softmax(x, dim=-1)：沿最后一维做 softmax
#   - dim=-1 等价于 dim=1，这里是每一行
#   - 将每个 query 对所有 key 的分数归一化为概率
# - softmax公式：
#   α[i, j] = exp(attn_scores[i, j]) / sum_k exp(attn_scores[i, k])
# - 输出 attn_weights.shape = [6, 6]
# - 每一行和为 1，表示 query i 对所有 key 的注意力分布

# -------------------------------------------------------------
# 3. 打印注意力权重矩阵
# -------------------------------------------------------------
print(attn_weights)
# - 输出示例：
# tensor([[0.11, 0.18, 0.17, 0.08, 0.07, 0.39],
#         [0.13, 0.21, 0.20, 0.09, 0.08, 0.29],
#         ...])
# - 每一行表示一个 query 对所有 key 的注意力权重
# - 这些权重用于对 value 做加权平均，得到上下文向量


tensor([[0.2098, 0.2006, 0.1981, 0.1242, 0.1220, 0.1452],
        [0.1385, 0.2379, 0.2333, 0.1240, 0.1082, 0.1581],
        [0.1390, 0.2369, 0.2326, 0.1242, 0.1108, 0.1565],
        [0.1435, 0.2074, 0.2046, 0.1462, 0.1263, 0.1720],
        [0.1526, 0.1958, 0.1975, 0.1367, 0.1879, 0.1295],
        [0.1385, 0.2184, 0.2128, 0.1420, 0.0988, 0.1896]])


- 快速验证每一行的值确实加起来等于1：


In [15]:
# ================================================================
# 验证注意力权重矩阵每行的和
# ================================================================

# -------------------------------------------------------------
# 1. 手动求第二行的和
# -------------------------------------------------------------
row_2_sum = sum([0.1385, 0.2379, 0.2333, 0.1240, 0.1082, 0.1581])
# - sum() 将列表中的数相加
# - 验证 softmax 归一化后，每行的和应该接近 1
print("Row 2 sum:", row_2_sum)
# 输出：Row 2 sum: 1.0 （或非常接近 1）

# -------------------------------------------------------------
# 2. 使用 PyTorch 验证注意力矩阵每行和
# -------------------------------------------------------------
# attn_weights.shape = [seq_len, seq_len] = [6, 6]
# dim=-1 表示沿最后一维求和，即每行求和
all_row_sums = attn_weights.sum(dim=-1)
print("All row sums:", all_row_sums)
# - 输出示例：
# tensor([1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000])
# - 每一行的和都为 1，这是 softmax 的性质
# - 用于后续计算上下文向量时保证加权平均正确


Row 2 sum: 1.0
All row sums: tensor([1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000])


- 应用之前的 **步骤3** 来计算所有上下文向量（context vectors）：


In [16]:
# ================================================================
# 计算所有 query token 的上下文向量（Context Vectors）
# ================================================================

# -------------------------------------------------------------
# 1. attn_weights
# -------------------------------------------------------------
# attn_weights.shape = [seq_len, seq_len] = [6, 6]
# - 每一行表示 query token 对所有 key 的注意力权重
# - 行 i: query i 对所有 key 的权重 α[i, :]
# - 列 j: key j 的权重 α[:, j]

# -------------------------------------------------------------
# 2. inputs
# -------------------------------------------------------------
# inputs.shape = [seq_len, embedding_dim] = [6, 3]
# - 每一行是 token embedding
# - 每一列是 embedding 的维度

# -------------------------------------------------------------
# 3. 矩阵乘法计算上下文向量
# -------------------------------------------------------------
all_context_vecs = attn_weights @ inputs
# - @ 是矩阵乘法
# - 公式：
#     context_vec_i = sum_j (α[i,j] * x_j)
# - 解释：
#     1) attn_weights[i, j] 是 query i 对 key j 的注意力权重
#     2) x_j = inputs[j] 是 key j 的 embedding
#     3) 对所有 j 做加权求和，得到 query i 的上下文向量
# - 输出 shape = [seq_len, embedding_dim] = [6, 3]
# - 每一行对应一个 query token 的上下文向量

# -------------------------------------------------------------
# 4. 打印上下文向量
# -------------------------------------------------------------
print(all_context_vecs)
# - 输出示例：
# tensor([[0.47, 0.53, 0.61],
#         [0.55, 0.62, 0.65],
#         ...])
# - 这些向量结合了整个序列的信息，是自注意力机制的核心输出


tensor([[0.4421, 0.5931, 0.5790],
        [0.4419, 0.6515, 0.5683],
        [0.4431, 0.6496, 0.5671],
        [0.4304, 0.6298, 0.5510],
        [0.4671, 0.5910, 0.5266],
        [0.4177, 0.6503, 0.5645]])


- 为了进行合理性检查，之前计算的上下文向量 $z^{(2)} = [0.4419, 0.6515, 0.5683]$ 可以在上表的第2行找到：


In [17]:
# ================================================================
# 打印之前单独计算的第 2 个 query 的上下文向量
# ================================================================

# context_vec_2 是之前使用循环和加权求和计算的第 2 个 query 的上下文向量
# all_context_vecs 是使用矩阵乘法一次性计算所有 query 的上下文向量
# 它们应该是一样的

print("Previous 2nd context vector:", context_vec_2)
# 输出示例：
# Previous 2nd context vector: tensor([0.55, 0.62, 0.65])
# - 向量长度等于 embedding_dim
# - 代表第 2 个 token 综合了整个序列的上下文信息


Previous 2nd context vector: tensor([0.4419, 0.6515, 0.5683])


## 3.4 使用可训练权重实现自注意力


- 一个概念性框架，展示本节中开发的自注意力机制如何融入本书和本章的整体内容与结构


<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch03_compressed/13.webp" width="400px">

### 3.4.1 逐步计算注意力权重


- 在本节中，我们将实现原始 Transformer 架构、GPT 模型以及大多数流行 LLM 中使用的自注意力机制
- 这种自注意力机制也称为“缩放点积注意力（scaled dot-product attention）”
- 总体思路与之前类似：
  - 我们希望针对某个输入元素计算上下文向量，即输入向量的加权和
  - 为此，需要计算注意力权重
- 如你所见，与之前介绍的基本注意力机制相比，只存在轻微差别：
  - 最显著的区别是引入了在模型训练过程中会更新的权重矩阵
  - 这些可训练的权重矩阵非常关键，使模型（特别是模型内部的注意力模块）能够学习生成“优秀”的上下文向量


<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch03_compressed/14.webp" width="600px">

- 在逐步实现自注意力机制时，我们首先引入三个训练权重矩阵 $W_q$、$W_k$ 和 $W_v$
- 这三个矩阵用于通过矩阵乘法将嵌入的输入 token $x^{(i)}$ 投影为查询向量（query）、键向量（key）和值向量（value）：

  - 查询向量：$q^{(i)} = x^{(i)}\,W_q$
  - 键向量：$k^{(i)} = x^{(i)}\,W_k$
  - 值向量：$v^{(i)} = x^{(i)}\,W_v$


- 输入 $x$ 与查询向量 $q$ 的嵌入维度可以相同也可以不同，这取决于模型的设计和具体实现
- 在 GPT 模型中，输入和输出维度通常相同，但为了说明计算过程，为了更好地理解，这里我们选择不同的输入和输出维度：


In [18]:
# ================================================================
# 定义输入向量和输出维度
# ================================================================

# -------------------------------------------------------------
# 1. 获取第二个输入 token 向量
# -------------------------------------------------------------
x_2 = inputs[1]
# - inputs.shape = [seq_len, embedding_dim] = [6, 3]
# - x_2: 第二个 token 的 embedding 向量
# - shape = [embedding_dim] = [3]
# - 举例：tensor([0.55, 0.87, 0.66])

# -------------------------------------------------------------
# 2. 输入维度 d_in
# -------------------------------------------------------------
d_in = inputs.shape[1]
# - 输入向量的维度（embedding_dim）
# - 对应每个 token 的特征长度
# - 这里 d_in = 3

# -------------------------------------------------------------
# 3. 输出维度 d_out
# -------------------------------------------------------------
d_out = 2
# - 假设我们想把输入向量映射到新的向量空间
# - 输出向量长度为 d_out = 2
# - 常用于线性变换或投影操作


- 下面初始化三个权重矩阵；注意这里将 `requires_grad=False`，以减少输出中的杂乱，仅用于说明。如果要用于模型训练，则应将 `requires_grad=True`，以便在训练过程中更新这些矩阵


In [19]:
# ================================================================
# 定义 Query、Key、Value 的投影矩阵
# ================================================================

# -------------------------------------------------------------
# 1. 设置随机种子
# -------------------------------------------------------------
torch.manual_seed(123)
# - 保证每次生成的随机数相同
# - 方便实验可复现

# -------------------------------------------------------------
# 2. 定义 Query 权重矩阵
# -------------------------------------------------------------
W_query = torch.nn.Parameter(
    torch.rand(d_in, d_out),  # 生成随机矩阵 shape = [d_in, d_out] = [3, 2]
    requires_grad=False        # 不进行梯度更新
)
# - 用于将输入向量映射到 Query 空间
# - 输入 x_2.shape = [d_in] → query = x_2 @ W_query
# - 结果 shape = [d_out] = [2]

# -------------------------------------------------------------
# 3. 定义 Key 权重矩阵
# -------------------------------------------------------------
W_key = torch.nn.Parameter(
    torch.rand(d_in, d_out),  # shape = [3, 2]
    requires_grad=False
)
# - 将输入向量映射到 Key 空间
# - 用于计算点积注意力

# -------------------------------------------------------------
# 4. 定义 Value 权重矩阵
# -------------------------------------------------------------
W_value = torch.nn.Parameter(
    torch.rand(d_in, d_out),  # shape = [3, 2]
    requires_grad=False
)
# - 将输入向量映射到 Value 空间
# - Value 是加权求和的对象，用于生成上下文向量


- 接下来，我们计算查询（query）、键（key）和值（value）向量：


In [20]:
# ================================================================
# 将第二个输入 token 投影到 Query、Key、Value 空间
# ================================================================

# -------------------------------------------------------------
# 1. query_2
# -------------------------------------------------------------
query_2 = x_2 @ W_query
# - x_2.shape = [d_in] = [3]
# - W_query.shape = [d_in, d_out] = [3, 2]
# - 矩阵乘法结果 shape = [d_out] = [2]
# - query_2 表示第二个 token 的 Query 向量
# - "_2" 表示对应第二个输入 token

# -------------------------------------------------------------
# 2. key_2
# -------------------------------------------------------------
key_2 = x_2 @ W_key
# - 输入向量映射到 Key 空间
# - shape = [d_out] = [2]

# -------------------------------------------------------------
# 3. value_2
# -------------------------------------------------------------
value_2 = x_2 @ W_value
# - 输入向量映射到 Value 空间
# - shape = [d_out] = [2]

# -------------------------------------------------------------
# 4. 打印 query_2
# -------------------------------------------------------------
print(query_2)
# - 输出示例：
# tensor([0.76, 1.20])
# - 这是第二个 token 在 Query 空间的表示，用于后续计算注意力分数


tensor([0.4306, 1.4551])


- 如下所示，我们成功地将6个输入标记从3维投影到2维嵌入空间：


In [21]:
# ================================================================
# 将整个输入序列投影到 Key 和 Value 空间
# ================================================================

# -------------------------------------------------------------
# 1. keys
# -------------------------------------------------------------
keys = inputs @ W_key
# - inputs.shape = [seq_len, d_in] = [6, 3]
# - W_key.shape = [d_in, d_out] = [3, 2]
# - 矩阵乘法结果 shape = [seq_len, d_out] = [6, 2]
# - 每一行 keys[i] 是第 i 个 token 的 Key 向量
# - 用于计算每个 query 对所有 token 的相似度

# -------------------------------------------------------------
# 2. values
# -------------------------------------------------------------
values = inputs @ W_value
# - shape = [seq_len, d_out] = [6, 2]
# - 每一行 values[i] 是第 i 个 token 的 Value 向量
# - 用于加权求和生成上下文向量

# -------------------------------------------------------------
# 3. 打印 shape
# -------------------------------------------------------------
print("keys.shape:", keys.shape)       # 输出: torch.Size([6, 2])
print("values.shape:", values.shape)   # 输出: torch.Size([6, 2])


keys.shape: torch.Size([6, 2])
values.shape: torch.Size([6, 2])


- 在下一步（**步骤2**）中，我们通过计算查询向量与每个键向量的点积来得到未归一化的注意力分数：


<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch03_compressed/15.webp" width="600px">

In [22]:
# ================================================================
# 计算第二个 query 对第二个 key 的注意力分数
# ================================================================

# -------------------------------------------------------------
# 1. 获取第二个 key 向量
# -------------------------------------------------------------
keys_2 = keys[1]
# - keys.shape = [seq_len, d_out] = [6, 2]
# - keys[1] 是第二个 token 的 Key 向量
# - Python 索引从 0 开始

# -------------------------------------------------------------
# 2. 计算注意力分数（点积）
# -------------------------------------------------------------
attn_score_22 = query_2.dot(keys_2)
# - query_2.shape = [d_out] = [2]
# - keys_2.shape  = [d_out] = [2]
# - torch.dot(a, b) 计算两个向量的点积：
#     attn_score_22 = sum(query_2[i] * keys_2[i] for i in range(d_out))
# - 结果是一个标量，表示 query_2 对 key_2 的相似度

# -------------------------------------------------------------
# 3. 打印注意力分数
# -------------------------------------------------------------
print(attn_score_22)
# - 输出示例：
# tensor(1.2345)
# - 这是自注意力机制中 query_2 对 key_2 的原始分数


tensor(1.8524)


- 由于我们有6个输入，对于给定的查询向量，我们将得到6个注意力分数：


In [23]:
# ================================================================
# 计算第二个 query 对所有 key 的注意力分数
# ================================================================

# -------------------------------------------------------------
# 1. query_2
# -------------------------------------------------------------
# shape = [d_out] = [2]
# - 对应第二个输入 token 的 Query 向量

# -------------------------------------------------------------
# 2. keys
# -------------------------------------------------------------
# shape = [seq_len, d_out] = [6, 2]
# - 每一行是序列中每个 token 的 Key 向量

# -------------------------------------------------------------
# 3. 矩阵乘法计算注意力分数
# -------------------------------------------------------------
attn_scores_2 = query_2 @ keys.T
# - keys.T.shape = [d_out, seq_len] = [2, 6]
# - query_2 @ keys.T → shape = [6]
# - 每个元素 attn_scores_2[j] = query_2 · keys[j]
#   - 点积表示 query 与 key 的相似度
# - 这是第二个 query 对所有 key 的注意力分数向量

# -------------------------------------------------------------
# 4. 打印结果
# -------------------------------------------------------------
print(attn_scores_2)
# - 输出示例：
# tensor([0.98, 1.23, 1.10, 0.75, 0.62, 0.89])
# - 这些分数用于 softmax 归一化生成注意力权重


tensor([1.2705, 1.8524, 1.8111, 1.0795, 0.5577, 1.5440])


<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch03_compressed/16.webp" width="600px">

- 接下来，在 **步骤3** 中，我们使用之前用过的 softmax 函数计算注意力权重（归一化的注意力分数，总和为1）  
- 与之前的不同之处在于，我们现在将注意力分数除以嵌入维度的平方根 $\sqrt{d_k}$（即 `d_k**0.5`）进行缩放：


In [24]:
# ================================================================
# 计算缩放点积注意力权重（Scaled Dot-Product Attention）
# ================================================================

# -------------------------------------------------------------
# 1. d_k: Key 向量维度
# -------------------------------------------------------------
d_k = keys.shape[1]
# - keys.shape = [seq_len, d_out] = [6, 2]
# - d_k = 2，对应 Key 向量的维度
# - 在缩放点积注意力中，需要除以 sqrt(d_k) 来防止点积值过大

# -------------------------------------------------------------
# 2. 缩放点积
# -------------------------------------------------------------
scaled_scores = attn_scores_2 / (d_k ** 0.5)
# - attn_scores_2.shape = [seq_len] = [6]
# - 除以 sqrt(d_k) 进行缩放，稳定梯度，避免 softmax 输出过于极端
# - 公式：scaled_score_j = attn_score_j / sqrt(d_k)

# -------------------------------------------------------------
# 3. softmax 归一化
# -------------------------------------------------------------
attn_weights_2 = torch.softmax(scaled_scores, dim=-1)
# - dim=-1 表示沿最后一维做 softmax
# - 每个元素 attn_weights_2[j] 表示 query_2 对 key_j 的注意力权重
# - 所有权重加和为 1
# - 输出 shape = [6]

# -------------------------------------------------------------
# 4. 打印注意力权重
# -------------------------------------------------------------
print(attn_weights_2)
# - 输出示例：
# tensor([0.2317, 0.2651, 0.2592, 0.1212, 0.1065, 0.0163])
# - 这些权重将用于对 value 向量加权求和生成上下文向量


tensor([0.1500, 0.2264, 0.2199, 0.1311, 0.0906, 0.1820])


<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch03_compressed/17.webp" width="600px">

- 在 **步骤4** 中，我们现在为输入查询向量2计算上下文向量（context vector）：


In [25]:
# ================================================================
# 计算第二个 query 的上下文向量（Context Vector）
# ================================================================

# -------------------------------------------------------------
# 1. attn_weights_2
# -------------------------------------------------------------
# shape = [seq_len] = [6]
# - 第二个 query 对所有 key 的注意力权重
# - 每个元素 α[j] 表示 query_2 对 key_j 的关注程度

# -------------------------------------------------------------
# 2. values
# -------------------------------------------------------------
# shape = [seq_len, d_out] = [6, 2]
# - 每一行是对应 token 的 Value 向量

# -------------------------------------------------------------
# 3. 矩阵乘法计算上下文向量
# -------------------------------------------------------------
context_vec_2 = attn_weights_2 @ values
# - @ 表示矩阵乘法
# - 公式：
#     context_vec_2 = sum_j (α[j] * values[j])
# - 解释：
#     1) 每个 value[j] 根据注意力权重 α[j] 加权
#     2) 得到 query_2 的上下文向量
# - 输出 shape = [d_out] = [2]

# -------------------------------------------------------------
# 4. 打印上下文向量
# -------------------------------------------------------------
print(context_vec_2)
# - 输出示例：
# tensor([0.49, 0.77])
# - 这个向量结合了第二个 token 与整个序列的信息
# - 是自注意力机制的核心输出


tensor([0.3061, 0.8210])


### 3.4.2 实现一个简洁的 SelfAttention 类


- 综合上述内容，我们可以如下实现自注意力机制：


In [28]:
# ================================================================
# 自注意力层（Self-Attention）v1：逐行逐参数详细注释
# ================================================================

import torch.nn as nn

class SelfAttention_v1(nn.Module):
    """
    单头自注意力层（Single-Head Self-Attention）

    输入：
        x: 输入序列张量，shape = [seq_len, d_in]
           seq_len = 序列长度（token 数）
           d_in = 每个 token 的 embedding 维度
    输出：
        context_vec: 上下文向量，shape = [seq_len, d_out]
           每个 token 的上下文向量通过对所有 token 的 Value 加权得到
    """

    def __init__(self, d_in, d_out):
        super().__init__()
        # -----------------------------------------------------
        # 定义 Query 权重矩阵
        # -----------------------------------------------------
        self.W_query = nn.Parameter(torch.rand(d_in, d_out))
        # - shape = [d_in, d_out]
        # - 用于将每个 token 的输入向量映射到 Query 空间
        # - 每个 token 的 query = x[i] @ W_query
        # - nn.Parameter 表示这是可训练参数（但这里未设置 requires_grad=False，所以默认可训练）

        # -----------------------------------------------------
        # 定义 Key 权重矩阵
        # -----------------------------------------------------
        self.W_key = nn.Parameter(torch.rand(d_in, d_out))
        # - shape = [d_in, d_out]
        # - 将输入映射到 Key 空间
        # - 用于计算注意力分数（query 与 key 的相似度）

        # -----------------------------------------------------
        # 定义 Value 权重矩阵
        # -----------------------------------------------------
        self.W_value = nn.Parameter(torch.rand(d_in, d_out))
        # - shape = [d_in, d_out]
        # - 将输入映射到 Value 空间
        # - Value 是加权求和的对象，生成每个 token 的上下文向量

    def forward(self, x):
        # -----------------------------------------------------
        # Step 1: 计算 Keys
        # -----------------------------------------------------
        keys = x @ self.W_key
        # - x.shape = [seq_len, d_in]
        # - W_key.shape = [d_in, d_out]
        # - 矩阵乘法结果 shape = [seq_len, d_out]
        # - keys[i] 是第 i 个 token 的 Key 向量，用于计算注意力分数

        # -----------------------------------------------------
        # Step 2: 计算 Queries
        # -----------------------------------------------------
        queries = x @ self.W_query
        # - shape = [seq_len, d_out]
        # - queries[i] 是第 i 个 token 的 Query 向量
        # - 用于计算与所有 key 的相似度

        # -----------------------------------------------------
        # Step 3: 计算 Values
        # -----------------------------------------------------
        values = x @ self.W_value
        # - shape = [seq_len, d_out]
        # - values[i] 是第 i 个 token 的 Value 向量
        # - 注意力加权后得到上下文向量

        # -----------------------------------------------------
        # Step 4: 计算注意力分数矩阵（omega）
        # -----------------------------------------------------
        attn_scores = queries @ keys.T
        # - queries.shape = [seq_len, d_out]
        # - keys.T.shape = [d_out, seq_len]
        # - 矩阵乘法 shape = [seq_len, seq_len]
        # - attn_scores[i, j] = queries[i] · keys[j]，表示第 i 个 token 对第 j 个 token 的注意力分数
        # - 每行表示对应 query 对所有 key 的分数

        # -----------------------------------------------------
        # Step 5: 缩放并归一化得到注意力权重
        # -----------------------------------------------------
        attn_weights = torch.softmax(
            attn_scores / (keys.shape[-1] ** 0.5),  # 除以 sqrt(d_k) 缩放
            dim=-1                                 # 对最后一维（每行）做 softmax
        )
        # - attn_weights.shape = [seq_len, seq_len]
        # - 每行和为 1，表示对应 query 对所有 token 的注意力分布

        # -----------------------------------------------------
        # Step 6: 加权求和得到上下文向量
        # -----------------------------------------------------
        context_vec = attn_weights @ values
        # - attn_weights.shape = [seq_len, seq_len]
        # - values.shape = [seq_len, d_out]
        # - 矩阵乘法 shape = [seq_len, d_out]
        # - 每行 context_vec[i] = sum_j(attn_weights[i,j] * values[j])
        # - 表示第 i 个 token 的上下文向量，融合了整个序列信息

        return context_vec

# ================================================================
# 测试 SelfAttention_v1
# ================================================================
torch.manual_seed(123)
sa_v1 = SelfAttention_v1(d_in, d_out)
# - d_in = 输入向量维度
# - d_out = 输出向量维度
# - 权重随机初始化，可复现

# 对输入序列计算上下文向量
output_context = sa_v1(inputs)
# - output_context.shape = [seq_len, d_out] = [6, 2]
# - 每个 token 的上下文向量已包含序列内信息

print(output_context)


tensor([[0.2996, 0.8053],
        [0.3061, 0.8210],
        [0.3058, 0.8203],
        [0.2948, 0.7939],
        [0.2927, 0.7891],
        [0.2990, 0.8040]], grad_fn=<MmBackward0>)


<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch03_compressed/18.webp" width="400px">

- 我们可以使用 PyTorch 的 Linear 层来简化上面的实现，如果禁用偏置项，Linear 层等价于矩阵乘法。
- 使用 `nn.Linear` 相比手动使用 `nn.Parameter(torch.rand(...))` 的另一个重要优势是，`nn.Linear` 有推荐的权重初始化方案，这有助于模型训练更加稳定。


In [29]:
# ================================================================
# 自注意力层（Self-Attention）v2：使用 nn.Linear 替代手动矩阵
# ================================================================

class SelfAttention_v2(nn.Module):
    """
    单头自注意力层（改进版）
    - 使用 nn.Linear 自动管理权重和可选偏置
    - 输入：x.shape = [seq_len, d_in]
    - 输出：context_vec.shape = [seq_len, d_out]
    """

    def __init__(self, d_in, d_out, qkv_bias=False):
        super().__init__()
        # -----------------------------------------------------
        # Query 映射层
        # -----------------------------------------------------
        self.W_query = nn.Linear(d_in, d_out, bias=qkv_bias)
        # - 将输入映射到 Query 空间
        # - nn.Linear 自动初始化权重矩阵 W_query.shape = [d_out, d_in]（内部转置）
        # - bias=qkv_bias 表示是否加偏置

        # -----------------------------------------------------
        # Key 映射层
        # -----------------------------------------------------
        self.W_key = nn.Linear(d_in, d_out, bias=qkv_bias)
        # - 将输入映射到 Key 空间

        # -----------------------------------------------------
        # Value 映射层
        # -----------------------------------------------------
        self.W_value = nn.Linear(d_in, d_out, bias=qkv_bias)
        # - 将输入映射到 Value 空间

    def forward(self, x):
        # -----------------------------------------------------
        # Step 1: 计算 Keys
        # -----------------------------------------------------
        keys = self.W_key(x)
        # - shape = [seq_len, d_out]
        # - 每个 token 的 Key 向量

        # -----------------------------------------------------
        # Step 2: 计算 Queries
        # -----------------------------------------------------
        queries = self.W_query(x)
        # - shape = [seq_len, d_out]
        # - 每个 token 的 Query 向量

        # -----------------------------------------------------
        # Step 3: 计算 Values
        # -----------------------------------------------------
        values = self.W_value(x)
        # - shape = [seq_len, d_out]
        # - 每个 token 的 Value 向量

        # -----------------------------------------------------
        # Step 4: 计算注意力分数
        # -----------------------------------------------------
        attn_scores = queries @ keys.T
        # - shape = [seq_len, seq_len]
        # - attn_scores[i, j] = queries[i] · keys[j]

        # -----------------------------------------------------
        # Step 5: 缩放并归一化得到注意力权重
        # -----------------------------------------------------
        attn_weights = torch.softmax(
            attn_scores / (keys.shape[-1] ** 0.5),
            dim=-1
        )
        # - dim=-1 表示对每行做 softmax
        # - 每行和为 1，表示对应 query 对所有 key 的注意力分布

        # -----------------------------------------------------
        # Step 6: 加权求和得到上下文向量
        # -----------------------------------------------------
        context_vec = attn_weights @ values
        # - shape = [seq_len, d_out]
        # - 每行是对应 query 的上下文向量
        # - 公式：context_vec[i] = sum_j(attn_weights[i,j] * values[j])

        return context_vec

# ================================================================
# 测试 SelfAttention_v2
# ================================================================
torch.manual_seed(789)  # 设置随机种子，保证权重初始化可复现
sa_v2 = SelfAttention_v2(d_in, d_out)
# - d_in = 输入向量维度
# - d_out = 输出向量维度

# 对输入序列计算上下文向量
output_context_v2 = sa_v2(inputs)
# - 输出 shape = [seq_len, d_out] = [6, 2]
# - 每个 token 的上下文向量融合了整个序列信息

print(output_context_v2)


tensor([[-0.0739,  0.0713],
        [-0.0748,  0.0703],
        [-0.0749,  0.0702],
        [-0.0760,  0.0685],
        [-0.0763,  0.0679],
        [-0.0754,  0.0693]], grad_fn=<MmBackward0>)


- 注意，`SelfAttention_v1` 和 `SelfAttention_v2` 的输出不同，因为它们使用了权重矩阵的不同初始化权重


## 3.5 使用因果注意力隐藏未来单词


- 在因果注意力（causal attention）中，对角线以上的注意力权重会被屏蔽，确保对于任何给定输入，LLM在计算上下文向量时无法使用未来的标记


<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch03_compressed/19.webp" width="400px">

### 3.5.1 应用因果注意力掩码


- 在本节中，我们将之前的自注意力机制转换为因果自注意力机制。
- 因果自注意力确保模型对序列中某个位置的预测仅依赖于前面已知的位置输出，而不依赖于未来位置。
- 简而言之，这保证了每个下一个词的预测只能依赖于前面的词。
- 为实现这一点，对于每个给定的 token，我们会屏蔽掉未来的 token（即输入文本中当前 token 之后的那些 token）：


<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch03_compressed/20.webp" width="600px">

- 为了说明并实现因果自注意力，我们使用上一节中的注意力分数和注意力权重：


In [30]:
# ===========================================================
# 复用上一节 SelfAttention_v2 对象中的 query 和 key 权重矩阵
# 以便演示注意力机制中的计算过程（不重新初始化新的网络）
# ===========================================================

# -----------------------------------------------------------
# 通过线性层 W_query 将输入 inputs 映射到“查询向量”（queries）
# -----------------------------------------------------------
# sa_v2.W_query：这是 SelfAttention_v2 类中定义的线性层 nn.Linear(d_in, d_out)
# inputs：形状为 [6, 3]，即 6 个 token，每个 token 的向量维度是 3
# 线性层作用：queries = inputs × W_query^T + bias（若 qkv_bias=False，则无偏置项）
# 输出 queries 形状为 [6, d_out]，即 [6, 2]
queries = sa_v2.W_query(inputs)

# -----------------------------------------------------------
# 同理，通过线性层 W_key 将输入 inputs 映射到“键向量”（keys）
# -----------------------------------------------------------
# W_key：线性层 nn.Linear(d_in, d_out)
# 作用：keys = inputs × W_key^T + bias
# 输出 keys 的形状同样为 [6, 2]
keys = sa_v2.W_key(inputs) 

# -----------------------------------------------------------
# 计算注意力分数矩阵（attention scores）
# -----------------------------------------------------------
# queries @ keys.T：矩阵相乘，表示计算每个 query 与所有 key 的相似度
# - queries 的形状为 [6, 2]
# - keys.T 的形状为 [2, 6]
# 因此结果 attn_scores 的形状为 [6, 6]
# 解释：第 i 行第 j 列的值表示第 i 个 query 与第 j 个 key 的点积得分
attn_scores = queries @ keys.T

# -----------------------------------------------------------
# 计算注意力权重矩阵（attention weights）
# -----------------------------------------------------------
# softmax：将得分标准化，使每一行的权重和为 1，便于作为加权平均的权重
# attn_scores / keys.shape[-1]**0.5：
#   - keys.shape[-1] 表示 key 向量的维度 d_k
#   - 除以 sqrt(d_k) 是“缩放点积注意力”的关键步骤，用于防止数值过大导致梯度消失
# dim=-1：
#   - 表示在最后一个维度上进行 softmax，也就是对每个 query 的所有 key 得分进行归一化
attn_weights = torch.softmax(attn_scores / keys.shape[-1]**0.5, dim=-1)

# -----------------------------------------------------------
# 打印注意力权重矩阵
# -----------------------------------------------------------
# 每一行表示一个 query 对所有 key 的注意力分布（权重和为 1）
print(attn_weights)


tensor([[0.1921, 0.1646, 0.1652, 0.1550, 0.1721, 0.1510],
        [0.2041, 0.1659, 0.1662, 0.1496, 0.1665, 0.1477],
        [0.2036, 0.1659, 0.1662, 0.1498, 0.1664, 0.1480],
        [0.1869, 0.1667, 0.1668, 0.1571, 0.1661, 0.1564],
        [0.1830, 0.1669, 0.1670, 0.1588, 0.1658, 0.1585],
        [0.1935, 0.1663, 0.1666, 0.1542, 0.1666, 0.1529]],
       grad_fn=<SoftmaxBackward0>)


- 屏蔽未来注意力权重的最简单方法是使用 PyTorch 的 `tril` 函数创建掩码，将主对角线及以下的元素设为1，主对角线以上的元素设为0：


In [31]:
# ===========================================================
# 构造一个简单的下三角掩码（mask），用于模拟“因果掩码”（causal mask）
# 在 Transformer 的自注意力（Self-Attention）中，mask 用于阻止模型
# 在预测当前词时看到未来词的信息。
# ===========================================================

# -----------------------------------------------------------
# 计算上下文序列长度（context_length）
# -----------------------------------------------------------
# attn_scores.shape[0]：注意力分数矩阵的行数，即序列中 token 的数量。
# 例如，如果 attn_scores 是一个 6×6 的矩阵，说明有 6 个 token。
context_length = attn_scores.shape[0]

# -----------------------------------------------------------
# 构造一个下三角矩阵（lower triangular matrix）
# -----------------------------------------------------------
# torch.ones(context_length, context_length)
#   → 创建一个全 1 矩阵，形状为 [6, 6]
#
# torch.tril(...) 
#   → “tril” 表示 “triangular lower”，即取矩阵的下三角部分（包括主对角线）。
#   → 主对角线以下（含主对角线）元素保留为 1；
#     主对角线上方的元素设为 0。
#   → 这样形成的掩码矩阵用于限制注意力计算：
#     位置 i 的 token 只能关注自己及之前的 token。
#
# 例如（context_length=6）：
# tensor([[1., 0., 0., 0., 0., 0.],
#         [1., 1., 0., 0., 0., 0.],
#         [1., 1., 1., 0., 0., 0.],
#         [1., 1., 1., 1., 0., 0.],
#         [1., 1., 1., 1., 1., 0.],
#         [1., 1., 1., 1., 1., 1.]])
mask_simple = torch.tril(torch.ones(context_length, context_length))

# -----------------------------------------------------------
# 打印掩码矩阵（mask_simple）
# -----------------------------------------------------------
# 输出结果展示每个位置能看到的范围。
# 例如第 3 行（从 0 开始计）只能看到前 3 个词（包括自己）。
print(mask_simple)


tensor([[1., 0., 0., 0., 0., 0.],
        [1., 1., 0., 0., 0., 0.],
        [1., 1., 1., 0., 0., 0.],
        [1., 1., 1., 1., 0., 0.],
        [1., 1., 1., 1., 1., 0.],
        [1., 1., 1., 1., 1., 1.]])


- 然后，我们可以将注意力权重与该掩码相乘，从而将对角线以上的注意力分数置为0：


In [32]:
# ===========================================================
# 将注意力权重矩阵（attn_weights）与下三角掩码（mask_simple）相乘
# 以“屏蔽”未来信息，确保模型在预测当前位置时
# 只能看到当前及之前的 token。
# ===========================================================

# -----------------------------------------------------------
# attn_weights：形状为 [context_length, context_length]
#   - 每一行表示一个 token（query）对所有 token（key）的注意力权重分布。
#   - 例如，第 3 行代表第 3 个词（query）对前 6 个词的关注程度。
#
# mask_simple：形状相同的下三角矩阵（0/1 矩阵）
#   - 值为 1 的地方表示允许关注；
#   - 值为 0 的地方表示不允许关注（即“未来词”）。
#
# 元素级相乘（逐元素相乘）：
#   attn_weights * mask_simple
#   → 所有 mask_simple 中为 0 的位置，对应的注意力权重被置为 0。
#
# 注意：这一步并没有重新归一化权重，因此每行的和可能不再是 1。
#       在实际的 Transformer 实现中，通常会对被 mask 的位置加上一个
#       “非常大的负数” (-inf)，然后再做 softmax。
#       这样 softmax 之后被掩码的部分权重自然会变为 0。
# -----------------------------------------------------------

masked_simple = attn_weights * mask_simple

# -----------------------------------------------------------
# 打印被掩码后的注意力权重矩阵
# -----------------------------------------------------------
# 输出说明：
# - 上三角（未来）部分的值应为 0。
# - 下三角（过去及当前）部分保留原有权重值。
print(masked_simple)


tensor([[0.1921, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2041, 0.1659, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2036, 0.1659, 0.1662, 0.0000, 0.0000, 0.0000],
        [0.1869, 0.1667, 0.1668, 0.1571, 0.0000, 0.0000],
        [0.1830, 0.1669, 0.1670, 0.1588, 0.1658, 0.0000],
        [0.1935, 0.1663, 0.1666, 0.1542, 0.1666, 0.1529]],
       grad_fn=<MulBackward0>)


- 但是，如果像上面那样在 softmax 之后再应用掩码，会破坏 softmax 创建的概率分布。
- softmax 确保所有输出值的和为 1。
- 在 softmax 之后进行掩码处理需要再次重新归一化输出，使其和为 1，这会使过程复杂化，并可能导致意外效果。


- 为确保每行的和为1，我们可以如下归一化注意力权重：


In [33]:
# ===========================================================
# 对被掩码的注意力权重矩阵重新进行归一化（Normalization）
# ===========================================================

# -----------------------------------------------------------
# masked_simple：被掩码后的注意力权重矩阵
#   - 经过 mask 处理后，上三角部分（未来信息）为 0。
#   - 但这样导致每一行的权重和不再为 1。
#   - 为了保持“注意力权重的概率意义”，我们需要对每一行重新归一化。
# -----------------------------------------------------------

# 计算每一行的总和（即当前 query 对所有允许的 key 的权重和）
# dim=-1 表示沿着最后一个维度（列方向）求和
# keepdim=True 保持二维张量形状（方便后续广播除法）
row_sums = masked_simple.sum(dim=-1, keepdim=True)

# -----------------------------------------------------------
# 每一行的权重除以该行总和，实现归一化：
# masked_simple_norm[i, j] = masked_simple[i, j] / row_sums[i]
# 这样可以确保每一行的权重之和重新归一化为 1。
# -----------------------------------------------------------
masked_simple_norm = masked_simple / row_sums

# -----------------------------------------------------------
# 打印结果
# -----------------------------------------------------------
# masked_simple_norm：
#   - 形状仍为 [context_length, context_length]
#   - 每一行现在的总和应接近 1。
print(masked_simple_norm)


tensor([[1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.5517, 0.4483, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.3800, 0.3097, 0.3103, 0.0000, 0.0000, 0.0000],
        [0.2758, 0.2460, 0.2462, 0.2319, 0.0000, 0.0000],
        [0.2175, 0.1983, 0.1984, 0.1888, 0.1971, 0.0000],
        [0.1935, 0.1663, 0.1666, 0.1542, 0.1666, 0.1529]],
       grad_fn=<DivBackward0>)


- 虽然我们从技术上已经完成了因果注意力机制的编码，但我们来简要看看一种更高效的方法，实现与上面相同的效果。
- 因此，不必在对角线以上将注意力权重置零并重新归一化结果，我们可以在进入 softmax 函数之前，将对角线以上的未归一化注意力分数掩码为负无穷：


<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch03_compressed/21.webp" width="450px">

In [34]:
# ===========================================================
# 使用上三角掩码（mask）实现注意力机制中的“因果遮掩”（Causal Masking）
# ===========================================================

# -----------------------------------------------------------
# 1️ 创建上三角矩阵（triu = triangular upper）
# -----------------------------------------------------------
# torch.ones(context_length, context_length)
#   - 创建一个大小为 [context_length, context_length] 的全 1 矩阵。
#   - 例如 context_length=6，则得到一个 6×6 的矩阵，所有元素均为 1。
#
# torch.triu(..., diagonal=1)
#   - 取出矩阵的上三角部分（包括主对角线以上的部分）。
#   - diagonal=1 表示“从主对角线往上移一格”开始保留上三角元素。
#   - 主对角线以下（包括主对角线）位置全部设为 0。
#
# 结果：
#   mask[i, j] = 1 如果 j > i（未来位置）
#   mask[i, j] = 0 如果 j ≤ i（当前或过去位置）
# -----------------------------------------------------------
mask = torch.triu(torch.ones(context_length, context_length), diagonal=1)

# -----------------------------------------------------------
# 2️ 应用掩码到注意力分数矩阵 attn_scores
# -----------------------------------------------------------
# attn_scores：形状 [context_length, context_length]
#   - 每个元素 attn_scores[i, j] 表示第 i 个 token 对第 j 个 token 的注意力分数。
#
# mask.bool()：
#   - 将 mask 中的 0/1 转换为布尔值（False/True），以便被 masked_fill 使用。
#
# masked_fill(mask.bool(), -torch.inf)：
#   - 对 mask 为 True（即 j > i，未来信息）的元素填充为 -∞。
#   - 这样在之后计算 softmax 时：
#       exp(-∞) ≈ 0
#     → 未来 token 的注意力权重被完全抑制。
# -----------------------------------------------------------
masked = attn_scores.masked_fill(mask.bool(), -torch.inf)

# -----------------------------------------------------------
# 3️ 打印结果
# -----------------------------------------------------------
# masked：
#   - 下三角部分保持原始 attn_scores。
#   - 上三角部分（未来时刻）为 -∞。
#   - 实现了 “decoder 只看当前和过去，不看未来” 的因果遮掩。
# -----------------------------------------------------------
print(masked)


tensor([[0.2899,   -inf,   -inf,   -inf,   -inf,   -inf],
        [0.4656, 0.1723,   -inf,   -inf,   -inf,   -inf],
        [0.4594, 0.1703, 0.1731,   -inf,   -inf,   -inf],
        [0.2642, 0.1024, 0.1036, 0.0186,   -inf,   -inf],
        [0.2183, 0.0874, 0.0882, 0.0177, 0.0786,   -inf],
        [0.3408, 0.1270, 0.1290, 0.0198, 0.1290, 0.0078]],
       grad_fn=<MaskedFillBackward0>)


- 如下所示，现在每行的注意力权重再次正确地加起来等于1：


In [35]:
# ===========================================================
# 计算“带掩码（mask）的注意力权重矩阵”
# ===========================================================

# -----------------------------------------------------------
# 1️ 对被掩码的注意力分数 masked 进行缩放
# -----------------------------------------------------------
# masked：
#   - 是上一步经过因果掩码（Causal Mask）处理后的注意力分数矩阵，
#   - 形状：[context_length, context_length]
#   - 其中未来位置（即 j > i）的分数被设为 -∞。
#
# keys.shape[-1]：
#   - 表示 key 向量的维度，也记作 d_k。
#   - 例如，如果每个 key 的维度是 2，则 keys.shape[-1] = 2。
#
# masked / keys.shape[-1]**0.5：
#   - “缩放点积注意力（Scaled Dot-Product Attention）” 的核心步骤。
#   - 除以 √d_k 目的是防止点积值过大导致 softmax 梯度过小（数值稳定性）。
# -----------------------------------------------------------

# -----------------------------------------------------------
# 2️ 通过 softmax 将注意力分数转换为注意力权重
# -----------------------------------------------------------
# torch.softmax(..., dim=-1)
#   - 对矩阵的每一行（即 dim=-1 表示最后一维）应用 softmax。
#   - softmax 公式：
#       softmax(x_i) = exp(x_i) / Σ_j exp(x_j)
#   - 每一行的元素（对应一个 query 的所有注意力分数）
#     被转换为 0~1 之间的概率分布。
#
# 注意：
#   - 由于被掩码部分是 -∞，exp(-∞) ≈ 0，
#     因此 softmax 之后这些位置的权重几乎为 0。
#   - 即，模型不会“看到未来”的 token。
# -----------------------------------------------------------
attn_weights = torch.softmax(masked / keys.shape[-1]**0.5, dim=-1)

# -----------------------------------------------------------
# 3️ 打印注意力权重矩阵
# -----------------------------------------------------------
# 每一行的和应当接近 1（浮点误差范围内），
# 且上三角部分（未来时刻）为 0。
# -----------------------------------------------------------
print(attn_weights)


tensor([[1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.5517, 0.4483, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.3800, 0.3097, 0.3103, 0.0000, 0.0000, 0.0000],
        [0.2758, 0.2460, 0.2462, 0.2319, 0.0000, 0.0000],
        [0.2175, 0.1983, 0.1984, 0.1888, 0.1971, 0.0000],
        [0.1935, 0.1663, 0.1666, 0.1542, 0.1666, 0.1529]],
       grad_fn=<SoftmaxBackward0>)


### 3.5.2 使用 dropout 屏蔽额外的注意力权重


- 此外，我们还会在训练过程中应用 dropout 以减少过拟合。
- Dropout 可以应用在多个位置：
  - 例如，在计算完注意力权重之后；
  - 或者在将注意力权重与值向量相乘之后。
- 在这里，我们将在计算完注意力权重后应用 dropout，因为这是更常见的做法。

- 此外，在这个特定示例中，我们使用 50% 的 dropout 率，这意味着随机屏蔽一半的注意力权重。（在后续训练 GPT 模型时，我们会使用较低的 dropout 率，例如 0.1 或 0.2）


<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch03_compressed/22.webp" width="400px">

- 如果我们应用 0.5（50%）的 dropout 率，未被丢弃的值将按 1/0.5 = 2 的比例进行缩放  
- 缩放的计算公式为 1 / (1 - `dropout_rate`)


In [36]:
# ===========================================================
# 演示 Dropout（随机失活）在注意力机制中的基本用法
# ===========================================================

# -----------------------------------------------------------
# 1️ 设置随机种子（保证结果可重复）
# -----------------------------------------------------------
# torch.manual_seed(123)：
#   设置 PyTorch 的随机数生成器种子。
#   这样每次运行本代码时，Dropout 随机丢弃的单元位置都相同，
#   方便教学和调试。
# -----------------------------------------------------------
torch.manual_seed(123)

# -----------------------------------------------------------
# 2️ 创建 Dropout 层
# -----------------------------------------------------------
# torch.nn.Dropout(p)
#   创建一个 Dropout 层，其中：
#     - p：表示丢弃的概率（dropout rate），范围 [0,1)
#     - 这里设为 0.5，表示每个元素有 50% 概率被“丢弃”
#
# Dropout 的作用：
#   - 在训练阶段，每次前向传播都会随机将部分神经元的输出设为 0。
#   - 这样可以防止模型过拟合（让模型不依赖某些特定特征）。
#   - 在推理（eval）模式下，Dropout 不会丢弃单元。
# -----------------------------------------------------------
dropout = torch.nn.Dropout(0.5)  # 丢弃率为 50%

# -----------------------------------------------------------
# 3️ 创建一个示例矩阵
# -----------------------------------------------------------
# torch.ones(6, 6)：
#   创建一个 6×6 的全 1 矩阵，表示模拟的输入特征。
# -----------------------------------------------------------
example = torch.ones(6, 6)

# -----------------------------------------------------------
# 4️ 对输入应用 Dropout
# -----------------------------------------------------------
# dropout(example)：
#   对 example 中的每个元素：
#     - 以 50% 概率将其值设为 0；
#     - 以 50% 概率保留，并按 (1 / (1 - p)) = 1 / 0.5 = 2 进行缩放。
#   原因：
#     这样在期望值意义上输出仍保持与输入相同的平均幅度。
#
# 输出结果：
#   - 形状仍为 [6, 6]；
#   - 约一半元素为 0；
#   - 另一半元素为 2（因为 1 * 1/(1-0.5) = 2）。
# -----------------------------------------------------------
print(dropout(example))


tensor([[2., 2., 0., 2., 2., 0.],
        [0., 0., 0., 2., 0., 2.],
        [2., 2., 2., 2., 0., 2.],
        [0., 2., 2., 0., 0., 2.],
        [0., 2., 0., 2., 0., 2.],
        [0., 2., 2., 2., 2., 0.]])


In [37]:
# ===========================================================
# 对注意力权重矩阵应用 Dropout（随机失活）示例
# ===========================================================

# -----------------------------------------------------------
# 1️ 固定随机数生成器种子
# -----------------------------------------------------------
# torch.manual_seed(123)
#   - 设置 PyTorch 的随机种子，保证每次运行时随机操作可复现。
#   - Dropout 内部使用随机数决定哪些元素被丢弃。
torch.manual_seed(123)

# -----------------------------------------------------------
# 2️ 对注意力权重矩阵应用 Dropout
# -----------------------------------------------------------
# dropout(attn_weights)
#   - attn_weights: 前面计算得到的注意力权重矩阵（softmax 输出），
#     形状为 [seq_len, seq_len]，每行元素之和约为 1。
#   - dropout: Dropout 层对象，之前定义为 torch.nn.Dropout(0.5)
#     - 参数 0.5 表示丢弃概率为 50%
#   - 工作原理：
#       1) 对每个元素，以 50% 的概率将其置为 0（被“丢弃”）
#       2) 被保留的元素会乘以 1/(1-0.5)=2，保证期望值不变
#   - 注意：Dropout 会破坏每行的和为 1 的性质，但训练时模型会适应这种噪声。
#
# 输出结果：
#   - 形状仍为 [seq_len, seq_len]
#   - 大约一半元素被置为 0，其余元素放大
print(dropout(attn_weights))


tensor([[2.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.7599, 0.6194, 0.6206, 0.0000, 0.0000, 0.0000],
        [0.0000, 0.4921, 0.4925, 0.0000, 0.0000, 0.0000],
        [0.0000, 0.3966, 0.0000, 0.3775, 0.0000, 0.0000],
        [0.0000, 0.3327, 0.3331, 0.3084, 0.3331, 0.0000]],
       grad_fn=<MulBackward0>)


- 请注意，生成的 dropout 输出可能因操作系统不同而有所差异；关于这种不一致性，可以在 [PyTorch 问题跟踪](https://github.com/pytorch/pytorch/issues/121595) 中了解更多


### 3.5.3 实现一个简洁的因果自注意力类


- 现在，我们已经准备好实现一个完整的自注意力机制，包括因果（causal）掩码和 dropout 掩码。
- 还有一点是需要实现处理批量输入的代码，以便我们的 `CausalAttention` 类支持由第 2 章数据加载器生成的批量输出。
- 为了简化起见，为了模拟这样的批量输入，我们将输入文本示例进行复制：


In [38]:
# ===========================================================
# 将多个输入序列堆叠成一个 batch 张量
# ===========================================================

# -----------------------------------------------------------
# 1️ torch.stack 用法
# -----------------------------------------------------------
# torch.stack(tensors, dim=0)
#   - tensors: 需要堆叠的张量序列（这里是两个 inputs 张量）
#   - dim: 指定沿着哪个维度堆叠
#     - dim=0 表示在第 0 维（最外层）增加一个新的 batch 维度
#   - 返回一个新的张量，shape 会多一个维度
#
# 示例：
#   - inputs: shape [seq_len, embedding_dim] = [6, 3]
#   - batch = torch.stack((inputs, inputs), dim=0)
#   - batch.shape = [2, 6, 3]
#     - 2: batch size
#     - 6: 每个序列的 token 数量
#     - 3: 每个 token 的 embedding 维度
# -----------------------------------------------------------
batch = torch.stack((inputs, inputs), dim=0)

# -----------------------------------------------------------
# 2️ 打印 batch 张量形状
# -----------------------------------------------------------
# 输出解释：
#   - 第 0 维: batch 大小 = 2
#   - 第 1 维: 每个输入序列的 token 数量 = 6
#   - 第 2 维: 每个 token 的 embedding 维度 = 3
# -----------------------------------------------------------
print(batch.shape)  # 2 inputs with 6 tokens each, and each token has embedding dimension 3


torch.Size([2, 6, 3])


In [40]:
# ===========================================================
# 定义“因果注意力（Causal Attention）”类
# ===========================================================

class CausalAttention(nn.Module):
    """
    因果注意力（Causal / Masked Self-Attention）模块
    说明：
      - 用于 Transformer 的解码器中，
      - 通过上三角掩码(mask)阻止模型“看到未来”token。
    """

    def __init__(self, d_in, d_out, context_length, dropout, qkv_bias=False):
        """
        初始化 CausalAttention 模块
        
        参数：
        - d_in: 输入 token embedding 的维度
        - d_out: 输出 token embedding 的维度
        - context_length: 支持的最大上下文长度（序列长度）
        - dropout: 注意力权重的 dropout 概率
        - qkv_bias: 是否在线性变换中使用 bias
        """
        super().__init__()
        self.d_out = d_out

        # -------------------------------------------------------
        # 线性变换矩阵，用于生成 query, key, value
        # -------------------------------------------------------
        self.W_query = nn.Linear(d_in, d_out, bias=qkv_bias)  # Q = x @ W_query + b
        self.W_key   = nn.Linear(d_in, d_out, bias=qkv_bias)  # K = x @ W_key + b
        self.W_value = nn.Linear(d_in, d_out, bias=qkv_bias)  # V = x @ W_value + b

        # -------------------------------------------------------
        # Dropout 层，用于随机丢弃注意力连接
        # -------------------------------------------------------
        self.dropout = nn.Dropout(dropout)

        # -------------------------------------------------------
        # 注册上三角掩码 buffer，用于阻止“未来 token”注意力
        # torch.triu(torch.ones(context_length, context_length), diagonal=1)
        #   - 上三角部分（不包括主对角线）为 1，表示未来 token
        #   - 下三角部分（包括主对角线）为 0，表示允许注意力
        # self.register_buffer:
        #   - 将 mask 注册为模型的一部分，但不会被训练（不是参数）
        # -------------------------------------------------------
        self.register_buffer(
            'mask', 
            torch.triu(torch.ones(context_length, context_length), diagonal=1)
        )

    def forward(self, x):
        """
        前向传播
        
        输入：
        - x: 输入 batch，形状 [batch_size, num_tokens, d_in]
        
        输出：
        - context_vec: 上下文向量，形状 [batch_size, num_tokens, d_out]
        """
        # -------------------------------------------------------
        # 获取 batch 大小、token 数量、输入维度
        # -------------------------------------------------------
        b, num_tokens, d_in = x.shape

        # -------------------------------------------------------
        # 生成 keys, queries, values
        # -------------------------------------------------------
        keys = self.W_key(x)       # [b, num_tokens, d_out]
        queries = self.W_query(x)  # [b, num_tokens, d_out]
        values = self.W_value(x)   # [b, num_tokens, d_out]

        # -------------------------------------------------------
        # 计算注意力分数（点积）
        # queries @ keys.transpose(1, 2)
        # - queries: [b, num_tokens, d_out]
        # - keys.transpose(1,2): [b, d_out, num_tokens]
        # - attn_scores: [b, num_tokens, num_tokens]
        # -------------------------------------------------------
        attn_scores = queries @ keys.transpose(1, 2)

        # -------------------------------------------------------
        # 应用因果掩码，阻止模型看到未来 token
        # masked_fill_ 是 in-place 操作
        # 仅对实际 token 数量范围内的 mask[:num_tokens, :num_tokens] 生效，是因为 batch 中的序列长度可能小于 context_length，如果不裁剪会出问题。
        # 被掩码位置的值设为 -∞
        # -------------------------------------------------------
        attn_scores.masked_fill_(
            self.mask.bool()[:num_tokens, :num_tokens], -torch.inf
        )

        # -------------------------------------------------------
        # 计算注意力权重
        # - 使用缩放点积注意力公式
        # - dim=-1 对每个 query 的注意力分布做 softmax
        # -------------------------------------------------------
        attn_weights = torch.softmax(
            attn_scores / keys.shape[-1]**0.5, dim=-1
        )

        # -------------------------------------------------------
        # 对注意力权重应用 dropout
        # -------------------------------------------------------
        attn_weights = self.dropout(attn_weights)

        # -------------------------------------------------------
        # 生成上下文向量
        # context_vec = attn_weights @ values
        # - attn_weights: [b, num_tokens, num_tokens]
        # - values: [b, num_tokens, d_out]
        # - 输出: [b, num_tokens, d_out]
        # -------------------------------------------------------
        context_vec = attn_weights @ values
        return context_vec

# -----------------------------------------------------------
# 设置随机种子，保证可复现
# -----------------------------------------------------------
torch.manual_seed(123)

# -----------------------------------------------------------
# context_length = 序列长度
# batch.shape = [batch_size, num_tokens, d_in]
# -----------------------------------------------------------
context_length = batch.shape[1]

# -----------------------------------------------------------
# 初始化 CausalAttention 对象
# dropout=0.0 表示不丢弃任何连接
# -----------------------------------------------------------
ca = CausalAttention(d_in, d_out, context_length, 0.0)

# -----------------------------------------------------------
# 前向传播，计算上下文向量
# context_vecs 形状: [batch_size, num_tokens, d_out]
# -----------------------------------------------------------
context_vecs = ca(batch)

# -----------------------------------------------------------
# 打印上下文向量及其形状
# -----------------------------------------------------------
print(context_vecs)
print("context_vecs.shape:", context_vecs.shape)


tensor([[[-0.4519,  0.2216],
         [-0.5874,  0.0058],
         [-0.6300, -0.0632],
         [-0.5675, -0.0843],
         [-0.5526, -0.0981],
         [-0.5299, -0.1081]],

        [[-0.4519,  0.2216],
         [-0.5874,  0.0058],
         [-0.6300, -0.0632],
         [-0.5675, -0.0843],
         [-0.5526, -0.0981],
         [-0.5299, -0.1081]]], grad_fn=<UnsafeViewBackward0>)
context_vecs.shape: torch.Size([2, 6, 2])


- 注意，dropout 仅在训练期间应用，在推理（inference）期间不使用


<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch03_compressed/23.webp" width="500px">

## 3.6 将单头注意力扩展到多头注意力


### 3.6.1 堆叠多个单头注意力层


- 下面是之前实现的自注意力机制的总结（为了简化，未显示因果掩码和 dropout 掩码）。

- 这也称为单头注意力（single-head attention）：

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch03_compressed/24.webp" width="400px">

- 我们只需堆叠多个单头注意力模块即可得到多头注意力模块：

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch03_compressed/25.webp" width="400px">

- 多头注意力的主要思想是对注意力机制进行多次（并行）运行，每次使用不同的、可学习的线性投影。这使得模型能够在不同的位置上联合关注来自不同表示子空间的信息。


In [41]:
# ===============================
# 定义多头注意力封装类
# ===============================
class MultiHeadAttentionWrapper(nn.Module):
    """
    MultiHeadAttentionWrapper 用于将多个独立的因果注意力头组合成多头注意力。
    """

    def __init__(self, d_in, d_out, context_length, dropout, num_heads, qkv_bias=False):
        """
        初始化多头注意力模块

        参数：
        - d_in: 输入 token embedding 的维度
        - d_out: 每个注意力头输出的维度
        - context_length: 序列最大长度（用于生成因果掩码）
        - dropout: 注意力权重 dropout 比例
        - num_heads: 注意力头数量
        - qkv_bias: 是否在线性层中使用偏置
        """
        super().__init__()  # 调用父类 nn.Module 的初始化

        # ---------------------------
        # 创建多个独立的 CausalAttention 注意力头
        # nn.ModuleList 用于存储子模块，PyTorch 会自动注册它们
        # 每个头的输入维度 d_in, 输出维度 d_out
        # ---------------------------
        self.heads = nn.ModuleList(
            [CausalAttention(d_in, d_out, context_length, dropout, qkv_bias) 
             for _ in range(num_heads)]  # 根据 num_heads 创建多个头
        )

    def forward(self, x):
        """
        前向传播函数

        参数：
        - x: 输入 tensor, 形状为 [batch_size, num_tokens, d_in]

        返回：
        - 多头注意力输出, 形状为 [batch_size, num_tokens, d_out*num_heads]
        """
        # 对每个注意力头计算上下文向量，并沿最后一维拼接
        # 例如: 两个头，每个头输出 [batch_size, num_tokens, d_out]
        # 拼接后变成 [batch_size, num_tokens, d_out*num_heads]
        return torch.cat([head(x) for head in self.heads], dim=-1)


# ===============================
# 测试 Multi-Head Attention
# ===============================
torch.manual_seed(123)  # 设置随机种子，保证可复现

context_length = batch.shape[1]  # 当前 batch 中 token 数量
d_in, d_out = 3, 2               # 输入维度 3, 每个头输出维度 2

# 创建多头注意力对象
mha = MultiHeadAttentionWrapper(
    d_in,                # 输入 embedding 维度
    d_out,               # 每个头输出维度
    context_length,      # 序列长度（用于掩码）
    0.0,                 # dropout = 0
    num_heads=2          # 两个注意力头
)

# 对 batch 进行多头注意力计算
context_vecs = mha(batch)  

# 打印上下文向量和形状
print(context_vecs)
print("context_vecs.shape:", context_vecs.shape)
# 输出形状: [batch_size, num_tokens, d_out*num_heads] = [2, 6, 4]


tensor([[[-0.4519,  0.2216,  0.4772,  0.1063],
         [-0.5874,  0.0058,  0.5891,  0.3257],
         [-0.6300, -0.0632,  0.6202,  0.3860],
         [-0.5675, -0.0843,  0.5478,  0.3589],
         [-0.5526, -0.0981,  0.5321,  0.3428],
         [-0.5299, -0.1081,  0.5077,  0.3493]],

        [[-0.4519,  0.2216,  0.4772,  0.1063],
         [-0.5874,  0.0058,  0.5891,  0.3257],
         [-0.6300, -0.0632,  0.6202,  0.3860],
         [-0.5675, -0.0843,  0.5478,  0.3589],
         [-0.5526, -0.0981,  0.5321,  0.3428],
         [-0.5299, -0.1081,  0.5077,  0.3493]]], grad_fn=<CatBackward0>)
context_vecs.shape: torch.Size([2, 6, 4])


- 在上述实现中，嵌入维度为4，因为我们将 `d_out=2` 作为键（key）、查询（query）、值（value）向量以及上下文向量（context vector）的嵌入维度。由于我们有2个注意力头（attention heads），输出嵌入维度为 2*2=4


### 3.6.2 使用权重拆分实现多头注意力


- 上述方法虽然是多头注意力的直观且完全可用的实现（封装了之前的单头注意力 `CausalAttention` 实现），但我们可以编写一个独立的类 `MultiHeadAttention` 来实现同样的功能。

- 对于这个独立的 `MultiHeadAttention` 类，我们不再简单地拼接单个注意力头。
- 相反，我们创建单个的 W_query、W_key 和 W_value 权重矩阵，然后将它们拆分为每个注意力头对应的独立矩阵：


In [42]:
# ===============================
# 多头注意力实现（带线性组合、因果掩码）
# ===============================

class MultiHeadAttention(nn.Module):
    """
    MultiHeadAttention 类：
    - 实现标准 Transformer 的多头自注意力机制
    - 支持 num_heads 个注意力头
    - 支持因果掩码（防止模型看到未来 token）
    - 支持线性组合多头输出
    """

    def __init__(self, d_in, d_out, context_length, dropout, num_heads, qkv_bias=False):
        """
        初始化多头注意力模块
        
        参数：
        - d_in: 输入 token embedding 的维度（例如 3）
        - d_out: 输出维度，总维度 = num_heads * head_dim
        - context_length: 序列的最大长度（用于因果掩码）
        - dropout: 注意力权重的 dropout 比例（0~1）
        - num_heads: 注意力头数量
        - qkv_bias: 是否在线性映射中使用偏置
        """
        super().__init__()

        # 检查 d_out 是否能被 num_heads 整除，否则每个 head 的维度无法计算
        assert (d_out % num_heads == 0), "d_out 必须能被 num_heads 整除"

        self.d_out = d_out                  # 总输出维度
        self.num_heads = num_heads          # 注意力头数量
        self.head_dim = d_out // num_heads  # 每个头的维度

        # ---------------------------
        # 定义线性层，用于生成 Q、K、V
        # 输入维度: d_in
        # 输出维度: d_out = num_heads * head_dim
        # ---------------------------
        self.W_query = nn.Linear(d_in, d_out, bias=qkv_bias)  # Q 矩阵
        self.W_key   = nn.Linear(d_in, d_out, bias=qkv_bias)  # K 矩阵
        self.W_value = nn.Linear(d_in, d_out, bias=qkv_bias)  # V 矩阵

        # 多头输出线性组合层，将所有头拼接后的向量映射回 d_out
        self.out_proj = nn.Linear(d_out, d_out)

        # dropout 用于正则化注意力权重，防止过拟合
        self.dropout = nn.Dropout(dropout)

        # ---------------------------
        # 因果掩码：上三角为 1，下三角（含主对角线）为 0
        # 用于防止注意力看到未来 token
        # mask 的 shape: [context_length, context_length]
        # ---------------------------
        self.register_buffer(
            "mask",
            torch.triu(torch.ones(context_length, context_length), diagonal=1)
        )

    def forward(self, x):
        """
        前向传播
        
        参数：
        - x: 输入 tensor，形状 [batch_size, num_tokens, d_in]

        返回：
        - context_vec: 上下文向量，形状 [batch_size, num_tokens, d_out]
        """
        # ---------------------------
        # 获取输入的 batch 大小、序列长度、输入维度
        # b: batch size
        # num_tokens: 序列长度
        # d_in: token embedding 维度
        # ---------------------------
        b, num_tokens, d_in = x.shape

        # ---------------------------
        # 线性映射生成 Q、K、V
        # shape: (b, num_tokens, d_out)
        # ---------------------------
        keys = self.W_key(x)
        queries = self.W_query(x)
        values = self.W_value(x)

        # ---------------------------
        # 拆分多头，将 d_out 拆成 num_heads * head_dim
        # 形状: (b, num_tokens, num_heads, head_dim)
        # ---------------------------
        keys = keys.view(b, num_tokens, self.num_heads, self.head_dim)
        values = values.view(b, num_tokens, self.num_heads, self.head_dim)
        queries = queries.view(b, num_tokens, self.num_heads, self.head_dim)

        # ---------------------------
        # 调整维度顺序，将 num_heads 放到第二维
        # 形状: (b, num_heads, num_tokens, head_dim)
        # ---------------------------
        keys = keys.transpose(1, 2)
        queries = queries.transpose(1, 2)
        values = values.transpose(1, 2)

        # ---------------------------
        # 计算缩放点积注意力
        # queries @ keys.transpose(2, 3)
        # queries: (b, num_heads, num_tokens, head_dim)
        # keys.transpose(2,3): (b, num_heads, head_dim, num_tokens)
        # attn_scores: (b, num_heads, num_tokens, num_tokens)
        # 每个 head 对每个 token 对应的注意力分数
        # ---------------------------
        attn_scores = queries @ keys.transpose(2, 3)

        # ---------------------------
        # 取 mask 的前 num_tokens 部分并转为 bool
        # 用于实际序列长度，不影响 padding 部分
        # ---------------------------
        mask_bool = self.mask.bool()[:num_tokens, :num_tokens]

        # ---------------------------
        # 使用因果掩码阻止模型看到未来 token
        # 被 mask 的位置置为 -inf，softmax 后权重接近 0
        # masked_fill_ 是 in-place 操作
        # ---------------------------
        attn_scores.masked_fill_(mask_bool, -torch.inf)

        # ---------------------------
        # softmax 缩放注意力权重
        # 除以 sqrt(head_dim) 防止梯度过大
        # shape: (b, num_heads, num_tokens, num_tokens)
        # ---------------------------
        attn_weights = torch.softmax(attn_scores / keys.shape[-1]**0.5, dim=-1)

        # dropout 正则化
        attn_weights = self.dropout(attn_weights)

        # ---------------------------
        # 应用注意力权重到 V
        # attn_weights @ values
        # shape: (b, num_heads, num_tokens, head_dim)
        # transpose(1,2) 调整维度顺序
        # shape: (b, num_tokens, num_heads, head_dim)
        # ---------------------------
        context_vec = (attn_weights @ values).transpose(1, 2)

        # ---------------------------
        # 拼接多个头
        # shape: (b, num_tokens, d_out)
        # ---------------------------
        context_vec = context_vec.contiguous().view(b, num_tokens, self.d_out)

        # ---------------------------
        # 可选线性组合，将拼接后的多头输出映射回 d_out
        # ---------------------------
        context_vec = self.out_proj(context_vec)

        return context_vec


# ===============================
# 测试 MultiHeadAttention
# ===============================
torch.manual_seed(123)

batch_size, context_length, d_in = batch.shape  # batch_size=2, context_length=6, d_in=3
d_out = 2                                       # 输出维度
mha = MultiHeadAttention(d_in, d_out, context_length, 0.0, num_heads=2)

# 前向传播得到上下文向量
context_vecs = mha(batch)

# 输出上下文向量及形状
print(context_vecs)
print("context_vecs.shape:", context_vecs.shape)
# 输出形状: [batch_size, num_tokens, d_out] = [2, 6, 2]


tensor([[[0.3190, 0.4858],
         [0.2943, 0.3897],
         [0.2856, 0.3593],
         [0.2693, 0.3873],
         [0.2639, 0.3928],
         [0.2575, 0.4028]],

        [[0.3190, 0.4858],
         [0.2943, 0.3897],
         [0.2856, 0.3593],
         [0.2693, 0.3873],
         [0.2639, 0.3928],
         [0.2575, 0.4028]]], grad_fn=<ViewBackward0>)
context_vecs.shape: torch.Size([2, 6, 2])


- 注意，上述内容本质上是对 `MultiHeadAttentionWrapper` 的重写版本，效率更高。
- 最终输出看起来可能略有不同，因为随机权重初始化不同，但这两种实现都是完全可用的，可以在接下来的章节中实现的 GPT 类中使用。


---

**关于输出维度的说明**

- 在上面的 `MultiHeadAttention` 中，我使用了 `d_out=2`，以保持与之前 `MultiHeadAttentionWrapper` 类相同的设置。
- `MultiHeadAttentionWrapper` 由于拼接操作，会返回输出头的维度为 `d_out * num_heads`（即 `2*2 = 4`）。
- 然而，`MultiHeadAttention` 类（为了更方便使用）允许我们通过 `d_out` 直接控制输出头的维度；这意味着，如果设置 `d_out = 2`，输出头的维度将为 2，而不受注意力头数量影响。
- 回顾来看，正如读者[指出](https://github.com/rasbt/LLMs-from-scratch/pull/859)，使用 `MultiHeadAttention` 时将 `d_out = 4` 可能更直观，这样可以产生与 `MultiHeadAttentionWrapper`（`d_out = 2`）相同的输出维度。

---


- 注意，此外我们在上面的 `MultiHeadAttention` 类中添加了一个线性投影层（`self.out_proj`）。这只是一个线性变换，并不会改变维度。在大型语言模型（LLM）实现中使用这样的投影层是标准惯例，但并非严格必要（近期研究表明，即使去掉它，也不会影响模型性能；详见本章末的进一步阅读部分）。


<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch03_compressed/26.webp" width="400px">

- 请注意，如果你希望实现上述功能的紧凑且高效的版本，也可以考虑使用 PyTorch 中的 [`torch.nn.MultiheadAttention`](https://pytorch.org/docs/stable/generated/torch.nn.MultiheadAttention.html) 类


- 由于上述实现乍一看可能有些复杂，我们来看执行 `attn_scores = queries @ keys.transpose(2, 3)` 时会发生什么：


In [43]:
# ---------------------------------------------
# 定义一个四维张量 a，模拟多头注意力中的注意力值
# 张量形状 (b, num_heads, num_tokens, head_dim) = (1, 2, 3, 4)
# - b = batch size = 1
# - num_heads = 注意力头数 = 2
# - num_tokens = token 数量 = 3
# - head_dim = 每个 head 的向量维度 = 4
# ---------------------------------------------
a = torch.tensor([[[[0.2745, 0.6584, 0.2775, 0.8573],   # 第 1 个 head，第 1 个 token 的向量
                    [0.8993, 0.0390, 0.9268, 0.7388],   # 第 1 个 head，第 2 个 token 的向量
                    [0.7179, 0.7058, 0.9156, 0.4340]],  # 第 1 个 head，第 3 个 token 的向量

                   [[0.0772, 0.3565, 0.1479, 0.5331],   # 第 2 个 head，第 1 个 token 的向量
                    [0.4066, 0.2318, 0.4545, 0.9737],   # 第 2 个 head，第 2 个 token 的向量
                    [0.4606, 0.5159, 0.4220, 0.5786]]]])# 第 2 个 head，第 3 个 token 的向量

# ---------------------------------------------
# a.transpose(2, 3) 解释：
# - 原张量形状： (b, num_heads, num_tokens, head_dim) = (1, 2, 3, 4)
# - transpose(2,3) 会交换第 2 维 (num_tokens) 与第 3 维 (head_dim)
# - 新形状： (1, 2, 4, 3)
# - 这是为了后续矩阵乘法，将 token 维与 head_dim 对齐
# ---------------------------------------------

# ---------------------------------------------
# 矩阵乘法 a @ a.transpose(2, 3)
# - a: (1, 2, 3, 4)
# - a.transpose(2,3): (1, 2, 4, 3)
# - 结果形状: (1, 2, 3, 3)
#   每个 head 中，3 个 token 与 3 个 token 的向量进行点积
# - 本质上是每个 head 中 token 向量与同一 head 中 token 向量的自相关矩阵
#   （类似 scaled dot-product 注意力中计算注意力分数）
# ---------------------------------------------
print(a @ a.transpose(2, 3))


tensor([[[[1.3208, 1.1631, 1.2879],
          [1.1631, 2.2150, 1.8424],
          [1.2879, 1.8424, 2.0402]],

         [[0.4391, 0.7003, 0.5903],
          [0.7003, 1.3737, 1.0620],
          [0.5903, 1.0620, 0.9912]]]])


- 在这种情况下，PyTorch 中的矩阵乘法实现会处理四维输入张量，使矩阵乘法在最后两个维度（num_tokens, head_dim）之间进行，并且对每个注意力头分别重复计算。

- 例如，下面这种方式可以更紧凑地对每个注意力头单独计算矩阵乘法：


In [44]:
# ---------------------------------------------
# 提取第一 batch 的两个 head 的数据，形状都是 (num_tokens, head_dim) = (3,4)
# ---------------------------------------------
first_head = a[0, 0, :, :]   # 第 1 个 head，形状 (3,4)
second_head = a[0, 1, :, :]  # 第 2 个 head，形状 (3,4)

# ---------------------------------------------
# 计算第一 head 内 token 的自相关矩阵
# - first_head @ first_head.T
# - first_head: (3,4)
# - first_head.T: (4,3)
# - 结果形状: (3,3)
#   每个元素表示 head 内 token 向量的点积
# ---------------------------------------------
first_res = first_head @ first_head.T
print("First head:\n", first_res)

# ---------------------------------------------
# 计算第二 head 内 token 的自相关矩阵
# - second_head @ second_head.T
# - second_head: (3,4)
# - second_head.T: (4,3)
# - 结果形状: (3,3)
# ---------------------------------------------
second_res = second_head @ second_head.T
print("\nSecond head:\n", second_res)


First head:
 tensor([[1.3208, 1.1631, 1.2879],
        [1.1631, 2.2150, 1.8424],
        [1.2879, 1.8424, 2.0402]])

Second head:
 tensor([[0.4391, 0.7003, 0.5903],
        [0.7003, 1.3737, 1.0620],
        [0.5903, 1.0620, 0.9912]])


# 总结与要点


- 请参见 [./multihead-attention.ipynb](./multihead-attention.ipynb) 代码笔记本，其中包含数据加载器（第2章）的简洁版本，以及本章实现的多头注意力类，这将在后续章节训练 GPT 模型时使用  
- 练习答案可以在 [./exercise-solutions.ipynb](./exercise-solutions.ipynb) 中找到
